# Run this before testing anything

In [201]:
import pandas as pd
import numpy as np
import json
import argparse
import os
from datetime import date
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import numpy as np

from src.selenium_manager import create_stealth_driver
from src.scraping import Scraper
from src.cleaning import DataCleaner, drop_mixed_listings, is_land_only
from src.feature_engineering import FeatureEngineer
from src.address_standardizer import AddressStandardizer
from src import config
from src.utils import save_urls_to_csv, save_details_to_csv, chunks
from src.tasks import scrape_worker
from src.modelling import predict_alley_width

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)

listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
cleaned = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')

def run_cleaning_pipeline():
    """Step 3: Clean the raw data and structure it."""
    if not os.path.exists(config.DETAILS_OUTPUT_FILE):
        print(f"Raw details file not found: {config.DETAILS_OUTPUT_FILE}. Run with `--mode details` first.")
        return

    print(f"Reading raw data from '{config.DETAILS_OUTPUT_FILE}'...")
    df_raw = pd.read_csv(config.DETAILS_OUTPUT_FILE)
    df_raw = drop_mixed_listings(df_raw)

    cleaned_records = []
    for _, row in df_raw.iterrows():
        row_dict = row.to_dict()
        direct_features = DataCleaner.extract_direct_features(row_dict)

        # --- 1. Detect if it is a land-only property ---
        is_land = is_land_only(row_dict)

        # --- 2. Extract all data ---
        processed_data = {
            'Tỉnh/Thành phố': DataCleaner.extract_city(row_dict),
            'Thành phố/Quận/Huyện/Thị xã': DataCleaner.extract_district(row_dict),
            'Xã/Phường/Thị trấn': DataCleaner.extract_ward(row_dict),
            'Đường phố': DataCleaner.extract_street(row_dict),
            'Chi tiết': DataCleaner.extract_address_detail(row_dict),
            'Nguồn thông tin': row_dict.get('url'),
            'Tình trạng giao dịch': 'Rao bán',
            'Thời điểm giao dịch/rao bán': DataCleaner.extract_published_date(row_dict.get('main_info')),
            'Thông tin liên hệ': None,
            'Giá rao bán/giao dịch': DataCleaner.extract_total_price(row_dict.get('main_info')),
            'Loại đơn giá (đ/m2 hoặc đ/m ngang)': 'đ/m2',
            'Số tầng công trình': DataCleaner.extract_num_floors(row_dict),
            'Tổng diện tích sàn': DataCleaner.extract_built_area(row_dict),
            'Đơn giá xây dựng': DataCleaner.get_construction_cost(row_dict),
            'Năm xây dựng': None,
            'Chất lượng còn lại': DataCleaner.estimate_remaining_quality(row_dict),
            'Diện tích đất (m2)': DataCleaner.extract_total_area(row_dict),
            'Kích thước mặt tiền (m)': DataCleaner.extract_facade_width(row_dict),
            'Kích thước chiều dài (m)': DataCleaner.extract_land_length(row_dict),
            'Số mặt tiền tiếp giáp': DataCleaner.extract_facade_count(row_dict),
            'Hình dạng': DataCleaner.extract_land_shape(row_dict),
            'Độ rộng ngõ/ngách nhỏ nhất (m)': DataCleaner.extract_alley_width(row_dict),
            'Khoảng cách tới trục đường chính (m)': DataCleaner.extract_distance_to_main_road(row_dict),
            'Mục đích sử dụng đất': 'Đất ở',
            'Yếu tố khác': " | ".join(direct_features) if direct_features else None,
            'Tọa độ (vĩ độ)': row_dict.get('latitude'),
            'Tọa độ (kinh độ)': row_dict.get('longitude'),
            'Hình ảnh của bài đăng': row_dict.get('image_urls'),
            'description': row_dict.get('description'),
            'is_land': is_land  # <-- Add the temporary flag here
        }

        # --- 3. Apply special logic if it's land only ---
        if is_land:
            processed_data['Số tầng công trình'] = 0
            processed_data['Đơn giá xây dựng'] = 0
            processed_data['Tổng diện tích sàn'] = 0
            processed_data['Chất lượng còn lại'] = 0
        
        cleaned_records.append(processed_data)

    df_cleaned = pd.DataFrame(cleaned_records)

    try:
        # Standardize Province and District using the simplified AddressStandardizer
        address_std = AddressStandardizer(
            config.PROVINCES_SQL_FILE,
            config.DISTRICTS_SQL_FILE,
            config.WARDS_SQL_FILE,
            config.STREETS_SQL_FILE
        )
        df_cleaned['Tỉnh/Thành phố'] = df_cleaned['Tỉnh/Thành phố'].apply(address_std.standardize_province)
        df_cleaned['short_address'] = df_raw['short_address']
        df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        df_cleaned['Xã/Phường/Thị trấn'] = df_cleaned.apply(address_std.standardize_ward, axis = 1)
        df_cleaned.drop(columns=['short_address'], inplace = True)
        df_cleaned.dropna(subset='Diện tích đất (m2)', axis=0, inplace=True)
        return df_cleaned
        # df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        # df_cleaned.drop(columns=['short_address'], inplace=True)
        # print("Province and District standardization complete.")
    except FileNotFoundError:
        print("Skipping province/district standardization because data files were not found.")

cleaned = run_cleaning_pipeline()

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)
cleaned.rename(columns={"Nguồn thông tin": 'url'}, inplace=True)

df = pd.merge(left=cleaned, right=listing_details, how='left', on='url')
df['other_info'] = df['other_info'].apply(json.loads)
df['main_info'] = df['main_info'].apply(json.loads)
# df = df[~(df['Diện tích đất (m2)'] == '')]
df.info()

Reading raw data from 'output/listing_details.csv'...
Removed 4972 listings containing 'thổ cư'.
Error parsing price: local variable 'cleaned_num' referenced before assignment
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27489 entries, 0 to 27488
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27489 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27489 non-null  object 
 2   Xã/Phường/Thị trấn                    27468 non-null  object 
 3   Đường phố                             23800 non-null  object 
 4   Chi tiết                              27489 non-null  object 
 5   url                                   27489 non-null  object 
 6   Tình trạng giao dịch                  27489 non-null  object 
 7   Thời điểm giao dịch/rao bán           27479 non-null  object 
 8   Thông tin liên hệ                     0 

In [117]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27489 entries, 0 to 27488
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27489 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27489 non-null  object 
 2   Xã/Phường/Thị trấn                    27467 non-null  object 
 3   Đường phố                             23800 non-null  object 
 4   Chi tiết                              27489 non-null  object 
 5   Nguồn thông tin                       27489 non-null  object 
 6   Tình trạng giao dịch                  27489 non-null  object 
 7   Thời điểm giao dịch/rao bán           27477 non-null  object 
 8   Thông tin liên hệ                     0 non-null      object 
 9   Giá rao bán/giao dịch                 24951 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27489 non-null  object 
 11  Số tầng công tr

# Columns to fix

- Thành phố/Quận/Huyện/Thị xã (Xong)
- Xã/Phường/Thị trấn (xong)
- Giá rao bán/giao dịch (Không check nữa)
- Diện tích đất (m2) (xong)
- Số tầng công trình (xong)
- Hình dạng (Đã thử sử dụng word embedding (sentence transformer) nhưng không thành công)
- Chất lượng còn lại (xong)
- Đường phố (xong)
- Chi tiết (xong)
- Đơn giá xây dựng (in progress)
- Tổng diện tích sàn (in progress)

Kích thước mặt tiền (m), Kích thước chiều dài (m), Số mặt tiền tiếp giáp, Độ rộng ngõ/ngách nhỏ nhất (m), Khoảng cách tới trục đường chính (m)

# Test district from here

In [9]:
import argparse
import os
from datetime import date
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import numpy as np

from src.selenium_manager import create_stealth_driver
from src.scraping import Scraper
from src.cleaning import DataCleaner, drop_mixed_listings, is_land_only
from src.feature_engineering import FeatureEngineer
from src.address_standardizer import AddressStandardizer
from src import config
from src.utils import save_urls_to_csv, save_details_to_csv, chunks
from src.tasks import scrape_worker
from src.modelling import predict_alley_width

def run_cleaning_pipeline():
    """Step 3: Clean the raw data and structure it."""
    if not os.path.exists(config.DETAILS_OUTPUT_FILE):
        print(f"Raw details file not found: {config.DETAILS_OUTPUT_FILE}. Run with `--mode details` first.")
        return

    print(f"Reading raw data from '{config.DETAILS_OUTPUT_FILE}'...")
    df_raw = pd.read_csv(config.DETAILS_OUTPUT_FILE)
    df_raw = drop_mixed_listings(df_raw)

    cleaned_records = []
    for _, row in df_raw.iterrows():
        row_dict = row.to_dict()
        direct_features = DataCleaner.extract_direct_features(row_dict)

        # --- 1. Detect if it is a land-only property ---
        is_land = is_land_only(row_dict)

        # --- 2. Extract all data ---
        processed_data = {
            'Tỉnh/Thành phố': DataCleaner.extract_city(row_dict),
            'Thành phố/Quận/Huyện/Thị xã': DataCleaner.extract_district(row_dict),
            'Xã/Phường/Thị trấn': DataCleaner.extract_ward(row_dict),
            'Đường phố': DataCleaner.extract_street(row_dict),
            'Chi tiết': DataCleaner.extract_address_detail(row_dict),
            'Nguồn thông tin': row_dict.get('url'),
            'Tình trạng giao dịch': 'Rao bán',
            'Thời điểm giao dịch/rao bán': DataCleaner.extract_published_date(row_dict.get('main_info')),
            'Thông tin liên hệ': None,
            'Giá rao bán/giao dịch': DataCleaner.extract_total_price(row_dict.get('main_info')),
            'Loại đơn giá (đ/m2 hoặc đ/m ngang)': 'đ/m2',
            'Số tầng công trình': DataCleaner.extract_num_floors(row_dict),
            'Tổng diện tích sàn': DataCleaner.extract_built_area(row_dict),
            'Đơn giá xây dựng': DataCleaner.get_construction_cost(row_dict),
            'Năm xây dựng': None,
            'Chất lượng còn lại': DataCleaner.estimate_remaining_quality(row_dict),
            'Diện tích đất (m2)': DataCleaner.extract_total_area(row_dict),
            'Kích thước mặt tiền (m)': DataCleaner.extract_facade_width(row_dict),
            'Kích thước chiều dài (m)': DataCleaner.extract_land_length(row_dict),
            'Số mặt tiền tiếp giáp': DataCleaner.extract_facade_count(row_dict),
            'Hình dạng': DataCleaner.extract_land_shape(row_dict),
            'Độ rộng ngõ/ngách nhỏ nhất (m)': DataCleaner.extract_alley_width(row_dict),
            'Khoảng cách tới trục đường chính (m)': DataCleaner.extract_distance_to_main_road(row_dict),
            'Mục đích sử dụng đất': 'Đất ở',
            'Yếu tố khác': " | ".join(direct_features) if direct_features else None,
            'Tọa độ (vĩ độ)': row_dict.get('latitude'),
            'Tọa độ (kinh độ)': row_dict.get('longitude'),
            'Hình ảnh của bài đăng': row_dict.get('image_urls'),
            'description': row_dict.get('description'),
            'is_land': is_land  # <-- Add the temporary flag here
        }

        # --- 3. Apply special logic if it's land only ---
        if is_land:
            processed_data['Số tầng công trình'] = 0
            processed_data['Đơn giá xây dựng'] = 0
            processed_data['Tổng diện tích sàn'] = 0
            processed_data['Chất lượng còn lại'] = 0
        
        cleaned_records.append(processed_data)

    df_cleaned = pd.DataFrame(cleaned_records)

    try:
        # Standardize Province and District using the simplified AddressStandardizer
        address_std = AddressStandardizer(
            config.PROVINCES_SQL_FILE,
            config.DISTRICTS_SQL_FILE,
            config.WARDS_SQL_FILE,
            config.STREETS_SQL_FILE
        )
        df_cleaned['Tỉnh/Thành phố'] = df_cleaned['Tỉnh/Thành phố'].apply(address_std.standardize_province)
        df_cleaned['short_address'] = df_raw['short_address']
        df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        df_cleaned['Xã/Phường/Thị trấn'] = df_cleaned.apply(address_std.standardize_ward, axis = 1)
        df_cleaned.drop(columns=['short_address'], inplace = True)
        return df_cleaned
        # df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        # df_cleaned.drop(columns=['short_address'], inplace=True)
        # print("Province and District standardization complete.")
    except FileNotFoundError:
        print("Skipping province/district standardization because data files were not found.")

cleaned = run_cleaning_pipeline()

Reading raw data from 'output/listing_details.csv'...
Removed 4972 listings containing 'thổ cư'.
Error parsing price: local variable 'cleaned_num' referenced before assignment


In [ ]:
# from src.address_standardizer import AddressStandardizer
from src import config
import sqlite3
import pandas as pd
from unicodedata import normalize
from rapidfuzz import fuzz

conn = sqlite3.connect(":memory:")
conn.execute("CREATE TABLE provinces (name TEXT, code TEXT, status TEXT);")
conn.execute("CREATE TABLE districts (name TEXT, code TEXT, province_code TEXT, status TEXT);")
conn.execute("CREATE TABLE wards (name TEXT, code TEXT, district_code TEXT, status TEXT);")
conn.execute("CREATE TABLE streets (name TEXT, code TEXT, district_code TEXT, status TEXT);")

with open(config.PROVINCES_SQL_FILE, "r", encoding="utf-8") as f:
    conn.executescript(f.read())

with open(config.DISTRICTS_SQL_FILE, "r", encoding="utf-8") as f:
    dis_cleaned = f.read().replace("\\'", "''")
    conn.executescript(dis_cleaned)

with open(config.WARDS_SQL_FILE, "r", encoding="utf-8") as f:
    ward_cleaned = f.read().replace("\\'", "''")
    conn.executescript(ward_cleaned)

provinces_df = pd.read_sql_query("SELECT * FROM provinces", conn)
districts_df = pd.read_sql_query("""
    SELECT d.name AS district_name, p.name AS province_name
    FROM districts d
    JOIN provinces p ON d.province_code = p.code
    """, conn)

wards_df = pd.read_sql_query("""
    SELECT w.name AS ward_name,
        d.name AS district_name,
        p.name AS province_name
    FROM wards w
    JOIN districts d ON w.district_code = d.code
    JOIN provinces p ON d.province_code = p.code
""", conn)

if 'conn' in locals():
    conn.close()

reverse_province_map = {
    prov.replace("Thành phố ", "").replace("Tỉnh ", ""): prov
    for prov in provinces_df['name'].unique()
}

reverse_district = {}
for province in districts_df['province_name'].unique():
    reverse_district[province] = {}
    for district_name in districts_df[districts_df['province_name'] == province]['district_name'].unique():
        district_name_strip = district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip()
        reverse_district[province][district_name_strip] = district_name
reverse_district['Tỉnh Bà Rịa - Vũng Tàu']['Long Đất'] = 'Huyện Long Đất'
reverse_district['Thành phố Hồ Chí Minh']['Quận 2'] = 'Thành phố Thủ Đức'
reverse_district['Thành phố Hồ Chí Minh']['Quận 9'] = 'Thành phố Thủ Đức'

# for province in districts_df['province_name'].unique():
#     reverse_district[province] = {}
#     for district_name in districts_df[districts_df['province_name'] == province]['district_name'].unique():
#         # district_name_strip = normalize('NFKD', district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip())
#         district_name_strip = district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip()
#         reverse_district[province][district_name_strip] = district_name
# reverse_district['Tỉnh Bà Rịa - Vũng Tàu']['Long Đất'] = 'Huyện Long Đất'

reverse_ward = {}
for province in reverse_district.keys():
    reverse_ward[province] = {}
    for district in reverse_district[province].values():
        reverse_ward[province][district] = {}
        for ward in wards_df[wards_df['district_name'] == district]['ward_name'].unique():
            ward_name_strip = normalize('NFC', ward.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '').replace('Thị Trấn ', '').strip())
            reverse_ward[province][district][ward_name_strip] = ward
# for district in wards_df['district_name'].unique():
#     reverse_ward[district] = {}
#     for ward_name in wards_df[wards_df['district_name'] == district]['ward_name'].unique():
#         ward_name_strip = normalize('NFKD', ward_name.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '').replace('Thị Trấn ', '').strip())
#         reverse_ward[district][ward_name_strip] = ward_name

def standardize_district(row):
        prefix = ['Thành phố', 'Thành Phố', 'Quận', 'Huyện', 'Thị xã', 'Thị Xã', 'Đảo']
        district_value = row['Thành phố/Quận/Huyện/Thị xã']
        if isinstance(district_value, str):
            for pre in prefix:
                if district_value.startswith(pre):
                    return district_value
            province = row['Tỉnh/Thành phố']
            if district_value in reverse_district[province].keys():
                return reverse_district[province][district_value]
            for dis in reverse_district[province].keys():
                similarity = fuzz.ratio(district_value, dis)
                if similarity >= 66:
                    print(f'Value: {district_value}')
                    print(f"Short address: {row['short_address']}")
                    print(f"Predicted value: {reverse_district[province][dis]}")
                    print('-' * 50)
                    return reverse_district[province][dis]
            return district_value
        return None

def standardize_ward(row):
        ward_value = row['Xã/Phường/Thị trấn']

        def matching(ward_value, district_value, province_value):
            # Function to match values with its corresponding prefixes
            try:
                if ward_value in reverse_ward[province_value][district_value].keys():
                    return reverse_ward[province_value][district_value][ward_value]
            except:
                print(f'Ward value: {ward_value}\nDistrict value: {district_value}\nProvince value: {province_value}')
                print('-'*50)
                return None
            for ward in reverse_ward[province_value][district_value].keys():
                similarity = fuzz.ratio(ward_value, ward)
                if similarity >= 66:
                    return reverse_ward[province_value][district_value][ward]
                
        if ward_value:
            prefix = ['Xã', 'Phường', 'Thị trấn', 'Thị Trấn']
            for pre in prefix:
                if ward_value.startswith(pre):
                    return ward_value
            ward_value = normalize('NFC', ward_value)
            province_value = row['Tỉnh/Thành phố']
            district_value = row['Thành phố/Quận/Huyện/Thị xã']
            return matching(ward_value, district_value, province_value)
        else:
            short_add_value = row['short_address']
            if isinstance(short_add_value, str) and short_add_value != '':
                short_add_list = row['short_address'].split(',')
                if len(short_add_list) >= 3:
                    new_province_val = row['Tỉnh/Thành phố']
                    new_ward_val = normalize('NFC',short_add_list[-3].strip())
                    new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                    return matching(new_ward_val, new_district_val, new_province_val)
            return None


# cleaned['ward'] = cleaned.apply(standardize_ward, axis=1)

# Test Ward

In [ ]:
from unicodedata import normalize
import re

def standardize_ward(row):
        ward_value = row['Xã/Phường/Thị trấn']

        def matching(ward_value, district_value, province_value):
            # Function to match values with its corresponding prefixes
            if ward_value in reverse_ward[province_value][district_value].keys():
                return reverse_ward[province_value][district_value][ward_value]
            for ward in reverse_ward[province_value][district_value].keys():
                similarity = fuzz.ratio(ward_value, ward)
                if similarity >= 66:
                    return reverse_ward[province_value][district_value][ward]
            return None
                
        if ward_value:
            prefix = ['Xã', 'Phường', 'Thị trấn', 'Thị Trấn']
            for pre in prefix:
                if ward_value.startswith(pre):
                    return ward_value
            ward_value = normalize('NFC', ward_value)
            province_value = row['Tỉnh/Thành phố']
            district_value = row['Thành phố/Quận/Huyện/Thị xã']
            return matching(ward_value, district_value, province_value)
        else:
            short_add = row['short_address']
            if isinstance(short_add, str) and short_add != '':
                if 'xã' in short_add.lower():
                    print(f"Xã in short_add: {short_add.lower()}")
                    match_result = re.search(pattern='(xã [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                elif 'phường' in short_add.lower():
                    print(f'Phường in short_add: {short_add.lower()}')
                    match_result = re.search(pattern='(phường [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                elif 'thị trấn' in short_add.lower():
                    print(f'Thị trấn in short_add: {short_add.lower()}')
                    match_result = re.search(pattern='(thị trấn [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                else:
                    short_add_list = row['short_address'].split(',')
                    if len(short_add_list) >= 3:
                        new_province_val = row['Tỉnh/Thành phố']
                        new_ward_val = normalize('NFC',short_add_list[-3].strip())
                        new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                        return matching(new_ward_val, new_district_val, new_province_val)

            else:
                return None
                # short_add_list = row['short_address'].split(',')
                # if len(short_add_list) >= 3:
                #     new_province_val = row['Tỉnh/Thành phố']
                #     new_ward_val = normalize('NFC',short_add_list[-3].strip())
                #     new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                #     return matching(new_ward_val, new_district_val, new_province_val)
            return None

In [ ]:
from rapidfuzz import fuzz

yo = cleaned['district'].iloc[24895]
dis = list(reverse_district['Tỉnh Đắk Lắk'].keys())[4]
print(fuzz.ratio(yo, dis))

66.66666666666667


# Test Prices (Mức giá)

In [ ]:
import pandas as pd
import numpy as np

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)
cleaned.rename(columns={"Nguồn thông tin": 'url'}, inplace=True)

df = pd.merge(left=cleaned, right=listing_details, how='left', on='url')
df.info()

In [ ]:
import json

df['other_info'] = df['other_info'].apply(json.loads)
df['price'] = df['other_info'].apply(lambda x: x.get('Mức giá'))

check_price_df = df[~df['price'].str.contains('tỷ', na=False)]
check_price_df = check_price_df[~(check_price_df['price'] == 'Thỏa thuận')]
print(f'Shape: {check_price_df.shape}')

check_price_df.dropna(subset = 'price', inplace=True, axis=0)
check_price_df['digit_price'] = check_price_df['price'].apply(lambda x: x.split()[0].replace(',', '.').strip())
check_price_df['digit_price'] = check_price_df['digit_price'].astype(float)
check_price_df['unit_price'] = check_price_df['price'].apply(lambda x: x.split()[1].strip())
print(check_price_df['unit_price'].unique())

nghin_met_vuong = check_price_df[check_price_df['unit_price'] == 'nghìn/m²']
nghin = check_price_df[check_price_df['unit_price'] == 'nghìn']
trieu_met_vuong = check_price_df[check_price_df['unit_price'] == 'triệu/m²']
trieu_incorrect = check_price_df[(check_price_df['unit_price'] == 'triệu') & (check_price_df['digit_price'] <= 300)]

# Test Khoảng cách tới trục đường chính

In [1]:
import pandas as pd
import json

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)

listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
df = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')

df['other_info'] = df['other_info'].apply(json.loads)
df['main_info'] = df['main_info'].apply(json.loads)

In [2]:
df['Đường vào'] = df['other_info'].apply(lambda x: x.get('Đường vào'))
df['Đường vào'].isna().sum()
df.drop_duplicates(subset='description', inplace=True)
df.dropna(subset='description', inplace=True)

In [82]:
import re

def extract_main_road_distance(des):
    if pd.notna(des):
        cach = re.findall(r'(cách\s*(?:\S+\s+){0,3}\d+(?:[.,]\d+)*m)', des.lower())
        ra = re.findall(r'(\d+(?:[.,]\d+)*m ra (?:\S+\s+){0,3})', des.lower())
        final = []
        if len(cach) > 0:
            final += cach
        if len(ra) > 0:
            final += ra
        if len(final) > 0:
            return str(final)
        if 'mặt phố' in des.lower() or 'mặt đường' in des.lower():
            return 'Mặt đường'
        return None
    return None

def new_extract_main_road_distance(des):
    if pd.notna(des):
        # ------TH1: Cách đường bao nhiêu m/bao nhiêu mét ra mặt đường------
        duong = 'đường|phố|mặt đường|mặt phố'
        mattien = 'mp|mặt tiền|mt|nhà|quốc lộ|ql'
        not_road = 'trường|chợ|siêu thị|vincom|aeon|lotte|biển|sông|hồ|bệnh viện|ubnd \
                    |công viên|cv|hẻm|hxh|ngõ|chung cư|cc|vườn|trung tâm|khu đô thị|kđt \
                    |kdt|vinmart|winmart|vin|mall|tttm|bigc|go|gigamall|sân bay|quận|q(?:\d+) \
                    |thành phố|tp|huyện|thị xã|thị trấn|tx'

        cach_duong_digit = re.search(rf'cách\s*(?:{duong})\s*(?:(?!\d+(?:[.,]\d+)*m)\S+\s*){{0,5}}?\D(\d+(?:[.,]\d+)*)m', des.lower())
        cach_not_digit_duong = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*m)(?!{not_road})\S+\s*){{0,3}}?\D(\d+(?:[.,]\d+)*)m\s*(?:(?!{not_road})\S+\s*){{0,2}}(?:{duong})\s*', des.lower())
        ra_duong = re.search(rf'\D(\d+(?:[.,]\d+)*)m\s*ra\s*(?:{duong})\s*(?:\S+\s+){{0,5}}', des.lower())
        cach_mattien_digit = re.search(rf'cách\s*(?:{mattien})\s*(?:(?!\d+(?:[.,]\d+)*m)(?!{not_road})\S+\s*){{0,5}}?\D(\d+(?:[.,]\d+)*)m', des.lower())
        cach_not_digit_mattien = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*m)(?!{not_road})\S+\s*){{0,3}}?\D(\d+(?:[.,]\d+)*)m\s*(?:(?!{not_road})\S+\s*){{0,2}}(?:{mattien})\s*', des.lower())
        ra_mattien = re.search(rf'\D(\d+(?:[.,]\d+)*)m\s*ra\s*(?:{mattien})\s*(?:\S+\s+){{0,5}}', des.lower())
        cach_not_digit_not_forbidden = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*m)(?!:{not_road})\S+\s*){{0,3}}\s*\D(\d+(?:[.,]\d+)*)m', des.lower())
        ra_digit_not_for = re.search(rf'\D(\d+(?:[,.]\d+)*)m\s*ra\s*(?:(?!{not_road})\S+\s+)', des.lower())

        all_patterns = [cach_duong_digit, cach_not_digit_duong, ra_duong, cach_mattien_digit, cach_not_digit_mattien, ra_mattien, cach_not_digit_not_forbidden, ra_digit_not_for]
        for pattern in all_patterns:
            if pattern:
                return float(pattern.group(1).replace(',', '.'))
        if 'mặt phố' in des.lower() or 'mặt đường' in des.lower():
            return 'Mặt đường'
        return None
    return None

def new_ver2_extract_main_road_distance(des):
    if pd.notna(des):
        # ------TH1: Cách đường bao nhiêu m/bao nhiêu mét ra mặt đường------
        # First, reject any description that contains a forbidden word near "cách"
        duong = 'đường|phố|mặt đường|mặt phố'
        mattien = 'mp|mặt tiền|mt|nhà|quốc lộ|ql|cầu'
        not_road = 'trường|chợ|siêu thị|vincom|aeon|lotte|biển|sông|hồ|bệnh viện|ubnd \
                    |công viên|cv|hẻm|hxh|ngõ|chung cư|cc|vườn|trung tâm|khu đô thị|kđt \
                    |kdt|vinmart|winmart|vin|mall|tttm|bigc|go|gigamall|sân bay|quận|q(?:\d+) \
                    |thành phố|tp|huyện|thị xã|thị trấn|tx'

        cach_duong_digit = re.search(rf'cách\s*(?:{duong})\s*(?:(?!\d+(?:[.,]\d+)*k*m)\S+\s*){{0,5}}?\D(\d+(?:[.,]\d+)*k*m)', des.lower())
        cach_not_digit_duong = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)\s*(?:(?!{not_road})\S+\s*){{0,2}}(?:{duong})\s*', des.lower())
        ra_duong = re.search(rf'\D(\d+(?:[.,]\d+)*k*m)\s*ra\s*(?:{duong})\s*(?:\S+\s+){{0,5}}', des.lower())
        cach_mattien_digit = re.search(rf'cách\s*(?:{mattien})\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)', des.lower())
        cach_not_digit_mattien = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)\s*(?:(?!{not_road})\S+\s*){{0,2}}(?:{mattien})\s*', des.lower())
        ra_mattien = re.search(rf'\D(\d+(?:[.,]\d+)*k*m)\s*ra\s*(?:{mattien})\s*(?:\S+\s+){{0,5}}', des.lower())
        cach_not_digit_not_forbidden = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}\D(\d+(?:[.,]\d+)*k*m)(?!\s*(?:\S+\s*){{0,7}}{not_road})', des.lower())
        ra_digit_not_for = re.search(rf'\D(\d+(?:[,.]\d+)*k*m)\s*ra\s*(?:(?!{not_road})\S+\s+)', des.lower())

        all_patterns = [cach_duong_digit, cach_not_digit_duong, ra_duong, cach_mattien_digit, cach_not_digit_mattien, ra_mattien, cach_not_digit_not_forbidden, ra_digit_not_for]
        for pattern in all_patterns:
            if pattern:
                return pattern.group(1).replace(',', '.')
        if 'mặt phố' in des.lower() or 'mặt đường' in des.lower():
            return 'Mặt đường'
        return None
    return None

df['main_road_distance'] = df['description'].apply(extract_main_road_distance)
df['new_main_road_distance'] = df['description'].apply(new_extract_main_road_distance)
df['new_ver2_main_road_distance'] = df['description'].apply(new_ver2_extract_main_road_distance)

In [103]:
df[(~df['new_ver2_main_road_distance'].isna()) & (df['new_ver2_main_road_distance'].str.contains('km'))].shape[0]

743

In [83]:
des = df.loc[57]['description']

duong = 'đường|phố|mặt đường|mặt phố'
mattien = 'mp|mặt tiền|mt|nhà|quốc lộ|ql|cầu'
not_road = 'trường|chợ|siêu thị|vincom|aeon|lotte|biển|sông|hồ|bệnh viện|ubnd \
            |công viên|cv|hẻm|hxh|ngõ|chung cư|cc|vườn|trung tâm|khu đô thị|kđt \
            |kdt|vinmart|winmart|vin|mall|tttm|bigc|go|gigamall|sân bay|quận|q(?:\d+) \
            |thành phố|tp|huyện|thị xã|thị trấn|tx'

# First, reject any description that contains a forbidden word near "cách"
if re.search(rf'cách[^.{{,;\n]]{{0,50}}{not_road}', des.lower()):
    print('No')

cach_duong_digit = re.search(rf'cách\s*(?:{duong})\s*(?:(?!\d+(?:[.,]\d+)*k*m)\S+\s*){{0,5}}?\D(\d+(?:[.,]\d+)*k*m)', des.lower())
cach_not_digit_duong = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)\s*(?:(?!{not_road})\S+\s*){{0,2}}(?:{duong})\s*', des.lower())
ra_duong = re.search(rf'\D(\d+(?:[.,]\d+)*k*m)\s*ra\s*(?:{duong})\s*(?:\S+\s+){{0,5}}', des.lower())
cach_mattien_digit = re.search(rf'cách\s*(?:{mattien})\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)', des.lower())
cach_not_digit_mattien = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)\s*(?:(?!{not_road})\S+\s*){{0,2}}(?:{mattien})\s*', des.lower())
ra_mattien = re.search(rf'\D(\d+(?:[.,]\d+)*k*m)\s*ra\s*(?:{mattien})\s*(?:\S+\s+){{0,5}}', des.lower())
cach_not_digit_not_forbidden = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)^/(?!{not_road})\S+\s*){{1,7}}\s*\D(\d+(?:[.,]\d+)*k*m)\s*(?:^/(?!{not_road})\S+\s*){{1,7}}', des.lower())
ra_digit_not_for = re.search(rf'\D(\d+(?:[,.]\d+)*k*m)\s*ra\s*(?:(?!{not_road})\S+\s+)', des.lower())

all_patterns = [cach_duong_digit, cach_not_digit_duong, ra_duong, cach_mattien_digit, cach_not_digit_mattien, ra_mattien, cach_not_digit_not_forbidden, ra_digit_not_for]

for pattern in all_patterns:
    print(pattern)

No
None
None
None
<re.Match object; span=(341, 367), match='cách cầu long biên hơn 1km'>
None
None
None
None


In [86]:
des = df.loc[749]['description']

duong = 'đường|phố|mặt đường|mặt phố'
mattien = 'mp|mặt tiền|mt|nhà|quốc lộ|ql'
not_road = 'trường|chợ|siêu thị|vincom|aeon|lotte|biển|sông|hồ|bệnh viện|ubnd \
            |công viên|cv|hẻm|hxh|ngõ|chung cư|cc|vườn|trung tâm|khu đô thị|kđt \
            |kdt|vinmart|winmart|vin|mall|tttm|bigc|go|gigamall|sân bay|quận|q(?:\d+) \
            |thành phố|tp|huyện|thị xã|thị trấn|tx'

cach_duong_digit = re.search(rf'cách\s*(?:{duong})\s*(?:(?!\d+(?:[.,]\d+)*k*m)\S+\s*){{0,5}}?\D(\d+(?:[.,]\d+)*k*m)', des.lower())
cach_not_digit_duong = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)\s*(?:(?!{not_road})\S+\s*){{0,2}}(?:{duong})\s*', des.lower())
ra_duong = re.search(rf'\D(\d+(?:[.,]\d+)*k*m)\s*ra\s*(?:{duong})\s*(?:\S+\s+){{0,5}}', des.lower())
cach_mattien_digit = re.search(rf'cách\s*(?:{mattien})\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)', des.lower())
cach_not_digit_mattien = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}?\D(\d+(?:[.,]\d+)*k*m)\s*(?:(?!{not_road})\S+\s*){{0,2}}(?:{mattien})\s*', des.lower())
ra_mattien = re.search(rf'\D(\d+(?:[.,]\d+)*k*m)\s*ra\s*(?:{mattien})\s*(?:\S+\s+){{0,5}}', des.lower())
cach_not_digit_not_forbidden = re.search(rf'cách\s*(?:(?!\d+(?:[.,]\d+)*k*m)(?!{not_road})\S+\s*){{1,7}}\D(\d+(?:[.,]\d+)*k*m)(?!\s*(?:\S+\s*){{0,7}}{not_road})', des.lower())
ra_digit_not_for = re.search(rf'\D(\d+(?:[,.]\d+)*k*m)\s*ra(?!\s*(?:\S+\s*){{0,7}}{not_road})', des.lower())
all_patterns = [cach_duong_digit, cach_not_digit_duong, ra_duong, cach_mattien_digit, cach_not_digit_mattien, ra_mattien, cach_not_digit_not_forbidden, ra_digit_not_for]

for pattern in all_patterns:
    print(pattern)

None
None
None
None
None
None
<re.Match object; span=(250, 262), match='cách hơn 1km'>
None


In [84]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

df[(df['new_ver2_main_road_distance'].str.contains('km')) & (~df['new_ver2_main_road_distance'].isna())][['new_ver2_main_road_distance','new_main_road_distance', 'description']]

,new_ver2_main_road_distance,new_main_road_distance,description
57,1km,None,"CỰC HIẾM - PHÂN LÔ NGỌC THUỴ - OTO TRÁNH - 7 TẦNG THANG MÁY - NỘI THẤT ĐẸP LONG LANH - AN NINH DÂN TRÍ CỰC CAO - KHU VỰC ÍT NHÀ BÁN\n\nVị Trí :\n- Nhà Nằm Trong Khu Vực Phân Lô Quân Đội An Ninh Dân Trí Cực Cao\n- Đường Trước Nhà 2 Oto Tránh Nhau Thoải Mái\n- Con Ngõ Đẹp Nhất Của Phường Ngọc Thuỵ\n- Cơ Sở Hạ Tầng Đồng Bộ , Chỉnh Chu , Sạch Đẹp\n- Cách Cầu Long Biên Hơn 1km Giao Thông Thuận Tiện , Vào Phố Chỉ 5 phút Chạy Xe\n- Không Khí Trong Lành Ở Sướng\n\nThông Số Nhà\n- Diện Tích Sổ 45m / Diện Tích Sử Dụng 50m\n- Mặt Tiền 4m = Hậu Vuông Đẹp\n\nNội Thất Đầy Đủ Chỉ Việc Mang Vali Về Ở\n- Thang Máy Nhập Khẩu 450kg\n- Giường , Tủ , Bàn Ghế Toàn Hàng Xịn\n\nThiết Kế :\nTầng 1: Gara ô tô + wc\nTầng 2: Phòng ngủ master + wc\nTầng 3: Phòng ngủ master + wc\nTầng 4: Phòng ngủ master + wc\nTầng 5: Phòng khách + wc\nTầng 6: phòng thờ + bếp + sân phơi\nTầng 7: sân BBQ\n\nSổ Đỏ Chính Chủ Sẵn Sàng Giao Dịch\n\nGiá Chỉ Loanh Quanh #13Tỷ Có Thương Lượng\n\nLiên Hệ Em Đức :\n0397 778 ***\n\n#LongBiên #chuongduong #ototranh #thangmay #ngọcthuy #ngọclâm #nhàdep"
263,5.5km,30.0,"- Cách Mỹ Đình 5,5km.\n- Bán nhà mới hoàn thiện, đầy đủ công năng, thiết kế hiện đại tại xã Kim Chung, huyện Hoài Đức, TP. Hà Nội.\n- Diện tích: 36m² x 4 tầng.\n- Ô tô đỗ cách nhà 10m, cách 30m là đường ô tô tránh, gần trục chính của xã.\n- Nhà gần trường, chợ, Quốc Lộ 32, Khu Đô thị Hinode Royal Park, giao thông thuận tiện, dân cư đông đúc, nhiều tiện ích...\n- Tầng 1: Phòng ngoài để xe, phòng trong gồm bếp và 1 WC.\n- Tầng 2: Thiết kế tầng lệch gồm: 1 phòng khách riêng cửa sổ rộng, 1 phòng ngủ có ban công lộng gió, 1 WC đầy đủ thiết bị vệ sinh, cầu thang gỗ Lim, bậc đá kim sa.\n- Tầng 3: 2 phòng ngủ đều có ban công, cửa sổ và 1 WC.\n- Tầng 4: Phòng ngủ, phòng thờ, phòng giặt, sân phơi, giếng trời thoáng mát.\n- Sổ đỏ chính chủ.\n- Quý khách có nhu cầu mời liên hệ:\n0334 993 ***\n."
384,2km,None,"Nhà riêng tọa lạc tại Quốc lộ 21B, Phường Phú Lãm, Hà Đông, Hà Nội, là lựa chọn lý tưởng cho những ai đang tìm kiếm một không gian sống tiện nghi và thoải mái. Với diện tích 31m², nhà được thiết kế hợp lý với 4 phòng ngủ và 3 phòng tắm, đáp ứng nhu cầu sinh hoạt của gia đình.\n\n- 4 phòng ngủ rộng rãi, tạo không gian nghỉ ngơi thoải mái.\n- 3 phòng tắm hiện đại, tiện lợi cho sinh hoạt hàng ngày.\n- Giá bán hấp dẫn: 3,15 tỷ VND, một cơ hội đầu tư tốt.\n- Pháp lý đầy đủ, đảm bảo an tâm cho người mua.\n\nƯu điểm:\n- Khu vực yên tĩnh, an ninh tốt, phù hợp với gia đình có trẻ nhỏ.\n- Thiết kế thông minh, tối ưu hóa không gian sử dụng.\n\n- Cách trường cao đẳng kinh tế - kỹ thuật thương mại chỉ 1km.\n- Gần các trường tiểu học Phú Lương 2 và Phú Lãm.\n- Siêu thị điện máy Pico cách nhà khoảng 2km, thuận tiện cho việc mua sắm.\n\nĐể biết thêm thông tin chi tiết, vui lòng liên hệ trịnh đông qua số điện thoại\n0962 298 ***\n."
395,1.5km,None,"- Số Nhà 3 thuộc khu tập thể betong Thăng Long, Cổ Điển, Hải Bối, Đông Anh, Hà Nội. Dân cư tấp nập, tiện ích đầy đủ bán kính 1km từ trường học, Bệnh viện, trung tâm TT, chợ...\n\n- Nhà 2 tầng đầy đủ công năng cho hộ gia đình ở. Đường trước nhà gần 4m, cách cầu Thăng Long 1.5km, đường Hoàng Sa 1km nên giao thông thuận tiện các ngả đường.\n\n- Sổ đỏ pháp lý chuẩn, sẵn sàng giao dịch trong ngày.\n\n- Giá chỉ 2.5 tỷ (thị trường ko có căn thứ 2)."
689,3km,None,"Hxh đường Vườn Lài, P. An Phú Đông, Quận 12.\nDT: 4x25m, 100m². Giá 5 - 6 tỷ tùy diện tích vị trí.\nKhu vực dân cư đang đông, cách Dương Quảng Hàm 2 - 3km.\nLh:\n0344 778 ***\n."
697,5km,None,"Bán nhà Liên Mạc Thượng Cát, ô tô đỗ cửa, ngõ to, 38m2x4t, nhiều ngủ, nhỉnh 5 tỷ, sát bãi ô tô.\n\n+ Lh:\n0969 686 ***\nem Quân zalo.\n\n+ Nhà nằm tại vị trí cực đẹp, ngõ ô tô chạy qua, cách đê Liên Mạc chỉ 50m, ngõ rộng, nhà nhiều ngủ.\n\n+ Gần nhà có bãi ô tô, trường học điểm các cấp, chợ, siêu thị, đường Tây Thăng Long.\n\n+ Kết nối Liên Mạc, Thượng Cát, Đại Cát, Đông Ngạc, Kẻ Vẽ, Tây Tựu, cách phố Nhổn 5km.\n\n+ Sổ đỏ chính chủ

Đúng:
- 57: cách cầu long biên hơn 1km
- 263: Cách Mỹ Đình 5,5km
- 395: cách cầu Thăng Long 1.5km
- 689: cách Dương Quảng Hàm 2 - 3km. 
- 697: cách phố Nhổn 5km (scrape theo m thì ta có cách đê Liên Mạc chỉ 50m)
- 714: cách phố nhổn 4km (scrape theo cái m thì sai vì nó lấy đoạn cách bãi ô tô chỉ 40m)
- 770: cách Phạm Văn Đồng tầm 1km
- 781: cách Tân Kỳ Tân Quý 1Km di chuyển
- 783: cách nhổn 1km
- 1143: Cách Vành Đai 4 chỉ 1km (có cả cách Đại Lộ Thăng Long 3.5km nhưng mà thôi vậy cũng được rồi)
- 1560: cách cầu Mai Lĩnh hơn 1km
- 1600: 1km ra Đại Học Đại Nam, gần ngay vành đai 4 (đúng do cái gần vành đai 4)
- 2096:  cách KĐT Đô Nghĩa chỉ hơn 2km, gần Vành Đai 4 (đúng là do cái gần vành đai 4)
- 2665: cách QL13, 2km
- 2816: cách Mỹ Đình chỉ 3km (scrape theo m trả về Mặt đường)
- 2847: Cách đường Trịnh Văn Bô 1.5km
- 2925:  Chỉ cách Cộng Hoà 1km
- 2962:  Cách cầu Chương Dương chỉ 1km
- 3159: Cách Ql 1A 1.2km
- 3309: Cách Quốc lộ 1A chưa tới 1km
- 3333:  7km ra tới ngã ba Ba La
- 3388: 3km ra đường 6
- 5075: 1.8km ra Quốc lộ 32, (theo m scrape sai vì cụm Bán kính 100m ra TH và THCS Kim Chung.)
- 5076: 1km ra Quốc lộ 32 (theo m scrape sai vì cụm Bán kính 100m ra TH và THCS Kim Chung.)
- 5701: Cách đường Phạm Văn Đồng 3 - 5KM
- 5769: nCách phố cổ chỉ 2km.
- 5959: Cách cầu thanh Trì chừng 1km
- 6234: Cách cầu chương dương 1,5km . 200m ra mipec . 500m ra chợ ngọc thuỵ

Sai: 
- 384:  Siêu thị điện máy Pico cách nhà khoảng 2km
- 521: Cách trung tâm hành chính mới 1km
- 557: cách Gigamall khoảng 1km. (sai lè vì Từ Phạm Văn Đồng vào đường 36 khoảng 500m, đường 36 vào nhà khoảng 50m)
- 749: cách hơn 1km là TTTM Aeon Mall Hà Đông
- 846: Cách KĐT Đô Nghĩa 1,5km
- 890: Cách trường THCS Vân Canh 1km.
- 1191: ách trường học tầm 1km, siêu thị t-mart cầu diễn cũng chỉ 1,5km
- 1475: Cách siêu thị Thành Đô, Đức Thành, và Thành Công khoảng 2km
- 1725: cách Q7 1km
- 1746: Cách sân bay Cam Ranh: 32km
- 1995:  Cách SVD Mỹ Đình 1km
- 2200: Cách trường học khoảng 1km
- 2245:  Cách trường mầm non Kinder Green 1km
- 2248:  cách trường học 1km
- 2327: Cách trường học khoảng 1km
- 2398: Cách trường Vinschool Thăng Long 1km
- 2958: Cách sân bay Cam Ranh: 38km.
- 3327: Cách khu đô thị GS chỉ 1km
- 3382: Vị trí đẹp cách Times City 1km trung tâm Hoàn Kiếm 6km
- 3407: Cách trường THPT Trưng Vương chỉ 1km
- 4126: Cách BigC Go Dĩ An chỉ 3km
- 4144: Cách Vinhomes Đan Phượng chỉ 2km (scrape theo m ra mặt đường do cụm mặt dường rộng)
- 4256: Cách Bến xe An Sương 2km
- 4449: nách Winmart + khoảng 1km
- 4458: Cách trường học Bill Gates 1km
- 4483: Cách trường mầm non Việt Mỹ 1km.
- 4666: Đường Huỳnh Châu Sổ quẹo vào hẻm 499 chỉ 80m, cách QL62 150km
- 4782: Cách trường học 1km
- 4871: Cách senco 5 2km (trường hợp này để yên cũng không có vấn đề gì)
- 4973: Nằm cách Quận 1 chỉ 2km
- 5008:  cách KCN Bắc Thăng Long chỉ 2km.
- 5251: cách trung tâm thành phố khoảng 2km
- 5336: Cách sân bay Cam Ranh: 33km.
- 5585: Cách Aeonmall Tân Phú 6km (scrape theo m là mặt đường)
- 5745: Cách tòa nhà hành chính 3km
- 5908: Cách trường THCS An Đà chỉ 1km (trong khi có Cách Lê Hồng Phong, phố Văn Cao khoảng 800m)
- 6027: cách sân bay 2km
- 6143: Cách trường học 1km, cách chợ/siêu thị 1,5km, công viên 2km
- 6282: cách sân vận động mỹ đình 2,5km
- 6626: Cách trường học tầm 2km
- 7024: cách Quận 1 khoảng 10 phút (2,9km)
- 7222: cách trung tâm Hà Nội chỉ 45km
- 7420:  Cách Quận 1 Chỉ 1km

Not defined:
- 3515: vào phố chỉ 5 phút .cách cầu chường dương chỉ 2km
- 4222: Cách vòng xoay An Lạc Bình Tân 2km.\nCách bến xe Miền Tây 3km
- 4496: Vị trí đẹp, nằm cạnh ga buýt nhanh BRT, đường Nguyễn Thanh Bình, Lê Trọng Tấn, 3km ra bến xe Yên Nghĩa
- 4667: Bệnh viện Đại Học Y cách nhà 1km, vài bước ra đường Trường Chinh (theo m thì ra mặt đường)
- 5325: Nhà nằm trong khu dân cư đường Đào Tông Nguyên thông ra đường Huỳnh Tấn Phát, thị trấn Nhà Bè, cách Quận 7 đúng 1,5km và cách Phú Mỹ Hưng chỉ 8 phút xe
- 5362: cách Quận 7 đúng 2km và cách Phú Mỹ Hưng chỉ 10 phút xe.
- 5421: cách quận 7 đúng 1,5km và cách Phú Mỹ Hưng chỉ 5 phút xe
- 5434: cách Quận 7 đúng 1,2km và cách Phú Mỹ Hưng chỉ 6 phút xe
- 5442: cách Quận 7 đúng 1,5km và cách Phú Mỹ Hưng chỉ 8 phút xe
- 5461: cách Quận 7 đúng 1,2km và cách Phú Mỹ Hưng chỉ 6 phút xe\
- 5479: cách Quận 7 đúng 1,2km và cách Phú Mỹ Hưng chỉ 7 phút xe
- 5566: cách Quận 7 đúng 1,3km và cách Phú Mỹ Hưng chỉ 8 phút xe.
- 5791: Vị trí đẹp, vài bước ra ngã 4 đèn đỏ Vạn Phúc Tố Hữu, giao thông thuận tiện, cách aeon mall hà đông 2km, cách làng lụa vạn phúc 1km

In [70]:
df[(~df['new_main_road_distance'].isna()) & (df['new_main_road_distance'] != 'Mặt đường')].shape[0]

5135

In [40]:
string = 'cách đường 30m hehe'
cach = re.findall(r'cách\s*(?:\S+\s+){0,2}(?:đường|phố|mp|nhà|mặt tiền)\s*(?:\S+\s*){0,5}?(\d+(?:[.,]\d+)*m)', string.lower())
ra = re.findall(r'(\d+(?:[.,]\d+)*)m\s*ra\s*(?:(?!\b(?:trường|chợ|siêu thị)\b)\S+\s*){0,5}', string.lower())
final = []
if len(cach) > 0:
    final += cach
if len(ra) > 0:
    final += ra
if len(final) > 0:
    print(final)

['30m']


Đúng: 
- 0 (cách đường 23/10 chỉ 70m)
- 3 (Bán nhà Phố)
- 6 (Bán nhà góc 2 mặt tiền đường)
- 13 (50m ra mặt phố Đại Mỗ)
- 14 (cách đường chính 20m)
- 17 (Bán nhà phố Huỳnh Thúc Kháng)
- 19 (cách đường chính 50m)
- 29 (Cần bán gấp căn nhà ngay mặt tiền đường Nguyễn Hữu Trí)
- 31 (ngõ rộng thoáng cách mặt phố 20m.)
- 34 (nhà cách mặt phố 10m)
- 35 (chỉ 5m ra đường)
- 43 (bán nhà ngay mặt tiền Lê Văn Sỹ)
- 51 (Căn hộ tập thể mặt phố)
- 52 (cách mặt tiền 20m)
- 53 ( cách mặt tiền 15m)
- 57 (Đường Trước Nhà 2 Oto Tránh Nhau Thoải Mái)
- 67 (Nhà phố hiện đại)
- 68 (Tương lai 2 mặt đường trước và sau)
- 69 (Mặt tiền đường Nguyễn Trung Nguyệ)
- 75 (Cần bán nhà phân lô phố Đỗ Quang, Cầu Giấy)
- 89 (Bán nhà Ngã Tư Sở - Thanh Xuân - ngõ ô tô chạy)
- 90 (Bán nhà Khương Trung - Thanh Xuân, vị trí siêu đẹp, ô tô tránh trước nhà)
- 94 (Nhà đẹp ở ngay trên đường Nguyễn Văn Hưởng)
- 102 (cách 20m ra mặt tiền Huỳnh Văn Bánh)
- 112 (Nhà rất gần mặt tiền đường lớn, cách khoảng 40m.)
- 119 (căn nhà phố)
- 123 (ngõ nông 10m)
- 130: Đường: Trục chính thảm nhựa.
- 131:  Đường trước nhà ô tô 7 chỗ tránh chạy thông, 20m ra trục mở rộng 20m vỉa hè
- 137:  500m ra phố cổ
- 143: BÁN NHÀ PHỐ GIẢI PHÓNG 
- 146: gõ rộng nông,
- 148: Bán Nhà Phân Lô Phố Ngọc Thụy, Mặt Phố Phân Lô 
- 166: Đường rộng rãi, ô tô 4 chỗ đỗ cửa hoặc vào gara thoải mái.
- 177: Phố hiếm nhà bán, tiện ích an sinh đỉnh, kinh doanh đỉnh, 6 tầng thang máy mới đẹp.
- 179: cực chất ở Trường Chinh
- 181: Nhà phố
- 191: cách 1 nhà ra mặt phố,
- 200:  nhà Đường Hoàng Việt, Phường 4
- 205: 20m ra phố.
- 218: Đường nhựa rộng 6m, vỉa hè mỗi bên 3m 
- 221: cách Tân Thới Hiệp 21 khoảng 40m
- 230: cách 50m đường Đào Tông Nguyên
- 237: cách 50m đường Đào Tông Nguyên
- 251:  chỉ cách 50m ra mặt phố, ô tô đỗ cổng
- 258: Bán nhà mặt tiền đường số 6
- 263: cách 30m là đường ô tô tránh, gần trục chính của xã
- 267:  Cách mặt phố 30m
- 271: Nhà cách mặt phố Võ Chí Công khoảng 100m,
- 289: Vị Trí: Khu Phân lô, vỉa hè, 2 ô tô tránh đỗ ngày đêm phố Trần Thái Tông thông Dịch Vọng Hậu, Duy Tân.
- 298: Bán nhà phố Hoa Bằng, Cầu Giấy - 2 mặt tiền 
- 314: cách mặt tiền 70m
- 336: 20m ra mặt đường
- 337: Bán nhà dân xây kiên cố Phú Diễn, phân lô quân đội, diện tích 48m x 5 tầng, mặt tiền 4m, giá chỉ 12 tỷ
- 338: đường Sư Vạn Hạnh tầm 100m
- 347: đường An Lộc tầm 300m.
- 348: Cách đường Trần Não khoảng 500m
- 349:  cách 10m ra ô tô tránh 
- 352: ngõ nông +  chỉ 5 phút là tới Hồ Tây, Lăng Bác, khu vực phố Cổ, Hồ Gươm
- 361: Tuyến đường huyết mạch, lượng lưu thông cực lớn tăng giá mạnh theo thời gian
- 364: 30m ra mặt phố Minh Khai
- 383: 40m ra mặt tiền đường 16

Sai: 
- 11 (0)
- 12 (scrape từ ngõ 850 đường Láng thành 850m trong khi nó có Nhà nằm vị trí cực đắc địa Phố Chùa Láng)
- 15 (scrape được 10m từ ngõ rộng)
- 16 (scrape từ /3 đường Huỳnh Văn Nghệ thành 3m)
- 21 (scrape được 10 từ ngõ nông gần phố mà trong đó có Nhà Mới Phố Kim Ngưu)
- 25 (scrape được 0 từ Ngay ngã 6 Phù Đổng. Ngay Nguyễn Trãi, Tôn Thất Tùng, Bùi Thị Xuân)
- 30 (scrape từ Ngõ to, rộng trong khi có Cách đường Lê Quang Đạo kéo dài chỉ 300m, cách UBND Đại Mỗ chỉ 100m)
- 32 (Ký Gửi Mua Bán Cho Thuê Nhà Phố Sài Gòn. --> thông tin cò)
- 33 (Nhà tiếp giáp mặt tiền đường nội bộ Dương Hiến Quyền rộng 7m ô tô lưu thông thoải mái.)
- 36 (Ngõ nông rộng thoáng, ô tô đỗ ngay nhà, cách phố chỉ vài bước chân --> 10)
- 42 (Ngõ vào 5m, thuận tiện cho xe ô tô tức là nó không phải ở mặt đường, nhưng scrape thành 0)
- 44 (Đường 5m khu dân trí, sát mặt tiền)
- 46 (ngõ nông cách 1 nhà ra đường oto tránh)
- 48 ( Cách đường Lê Quang Đạo kéo dài chỉ 300m mà scrape thành 10 do ngõ to, rộng)
- 59 ( cách mp Nguyễn Phúc Lai 2m mà scrape thành 0)
- 62 (scrape ra 0 không biết ở đâu)
- 66 (Diện tích: Ngang 6.7m --> 67m)
- 71 (oto đỗ ngay mặt phố "cách nhà 15m" nhưng scrape thành 0 do cụm mặt ngõ?)
- 73 ( bán nhà Nguyễn Sơn mà scrape được 7)
- 79 (Vị trí siêu vip nằm trong ngõ Hoàng Quốc Việt, ngay chợ Nghĩa Tân mà scrape được 0)
- 83 (cách 1 nhà ra đường ô tô, thoải mái cho xe vào mà scrape thành 1)
- 87 (không có gì nhưng scrape thành 2)
- 96 (Ngõ nông, rộng, thông, thoáng, ô tô vào nhà)
- 106 ( trước nhà 50m ra đường lớn Dương Quảng Hàm)
- 108 (scrape là 0 nhưng chẳng có cái gì chứng minh cả)
- 110 (ị trí: Thuộc ngõ 26 Tư Đình, ngõ rộng ô tô tránh, sạch đẹp. Cầu Trần Hưng Đạo chuẩn bị khởi công xây, tiềm năng giá trị tăng cao, 10p di chuyển ra cầu Vĩnh Tuy, Cầu Chương Dương sang Phố cực tiện --> scrape ra 10)
- 113 ( Ngõ thẳng rộng, ô tô 4 chỗ tránh nhau, dân trí cao, văn minh, ở đảm bảo sướng --> nhưng vẫn scrape ra 0)
- 115 (Nhà nằm gần ngã tư Nguyễn Chí Thanh, Huỳnh Thúc Kháng, Nguyên Hồng. --> scrape ra 0)
- 116
- 117 (đường to rộng 2 ô tô tránh nhau)
- 122 (Ngõ vào rộng 6m --> scrape thành 0)
- 125: Cách mặt đường 422B chỉ 100m. (scrape thành 422)
- 135: Ngõ vào rộng 16m, thuận tiện cho ô tô di chuyển.(scrape thành 0)
- 141: Ngõ vào rộng 12m, thuận tiện cho xe ô tô ra vào dễ dàng (scrape thành 0 do cụm Nhà riêng tại phố Lý Thường Kiệt)
- 149:  Mặt tiền 8m, ngõ vào rộng 10m, thuận tiện cho xe hơi (scrape ra 0)
- 153: Sân ngõ rộng 3m, 20m ra ô tô tránh, chợ, bãi đỗ xe. 300m ra Aeon Long Biên, cầu Vĩnh Tuy sang phố rất thuận tiện. (scrape ra 0)
- 157:  mặt ngõ thông ô tô xanh SM vào nhà - mặt tiền 5M siêu đẹp - 60M2 - 14 tỷ - kinh doanh đỉnh.\n\n- Giáp danh với nhiều trục đường chính
- 159: Ngõ rộng 3m, 100m ra Phố (scrape ra 10)
- 168: HXH 8m đường Bình Giã (hẻm xe hơi) --> scrape thành 8
- 173: cách 30m là mặt tiền đường số 2 Trường Thọ (scrape thành 0)
- 176: Bán nhà hẻm 11 đường Số 16 Phường Bình Hưng Hòa Quận Bình Tân. Khu vực dân trí cao, an ninh tốt đầy đủ tiện ích... Đường trước nhà 6m xe hơi ngủ trong nhà. (scrape thành 11)
- 178: 500m Quốc Lộ 1K (scrape thành 2)
- 192: Căn nhà toạ lạc phố Trần Quốc Vượng (scrape thành 10)
- 197: mặt ngõ kinh doanh nhỏ (scrape thành 0)
- 226: mặt tiền đường Ngô Quyền (do scrape phải đoạn thông 2 đầu đường ra đường Ngô Quyền và Khổng Tử)
- 252: Đường trước nhà rộng 7m (nhưng scrape thành 154 do nó là ngõ 154)
- 257: cách Phạm Văn Đồng chỉ 200m (mà scrape thành 0 do nhà phố chuẩn đẹp)
- 276: mặt tiền Hoàng Diệu - cách mặt tiền 30 m và cách mặt tiền Hoàng Diệu 30 m (scrape thành 2)
- 278: 15m ra ô tô (scrape thành 10)
- 321: oto tránh nhau cách nhà 50m (scrape thành 0)
- 323: cách ô tô tránh nhau chỉ 10m
- 325: đường Nguyễn Phúc Chu quẹo vô đường Hoàng Bật Đạt tầm 200m.
- 326: đúng 500m ra đường Vũ Trọng Khánh
- 333: cách đường Lê Trọng Tấn 400m (scrape thành 3)
- Bán nhà ngay mặt tiền họ Lê Tân Phú (scrape thành 19)
- 339: Cách mặt tiền Ung Văn Khiêm chỉ 100m (scrape thành 8)
- 343: cách Điện Biên Phủ & Võ Thị Sáu chỉ 100m (scrape thành 0)
- 346:  cách mặt tiền 1 căn, siêu xe tải lưu thông, thuận tiện di chuyển đến các khu vực trong thành phố (scrape thành 1)
- 373:  1 phút ra mặt phố Bạch Mai, ngõ nông thông rộng
- 396: 30m ra mặt phố
- 397: 20m ra mặt phố Xã Đàn
- 401:  20m ra mặt phố
- 405: Cần bán nhà đường 17B,
- 413: Bán nhà phố Nhật Tảo 
- 425: cách Hồng Tiến chưa đầy 100m (scrape thành 10)
- 428: cách mặt tiền chỉ 20m (scrape thành 6)
- 438: Phố Phố Vọng gần Trần Đại Nghĩa, Giải Phóng, Đại La, Trương Định, Kim Đồng.\n- Khu vực trung tâm, thuận tiện đi các quận nội thành, Hai Bà Trưng, Hà Đông, Thanh Xuân, Đống Đa, Hoàng Mai.\n- Vị trí nhà đẹp, khu phân lô, vỉa hè, ô tô tránh, nhà đẹp ở ngay, trung tâm, kinh doanh tốt.

Unknown:
- 124 (ô tô nhỏ đỗ cửa)
- 180: scrape thành 0 nhưng thực ra không có cái gì indicate
- 188: Khu vực hiếm nhà bán, mặt tiền rộng, phù hợp làm homestay, apartment, hoặc chia 2 3 lô thì đẹp hết ý, kinh doanh đỉnh\n\nLiên hệ: Tôi - Liên Nhà Phố:\n0977 097 ...
- 196
- 198: Hẻm 8m xe hơi tránh nhau (scrape thành 0 do Nhà phố 4 tầng)
- 202: Nhà hẻm 6m oto Quang Trung, P8 Gò Vấp, sát chợ Hạnh Thông Tây, cách mặt tiền 50m (scrape 50m)
- 204: Khu vực hiếm nhà bán, nhà đã xây mới đẹp, đường ô tô tránh, thang máy, nở hậu, cho thuê 50 triệu/ tháng.\n\nLiên hệ: Tôi - Liên nhà phố
- 211: Ngõ vào 2m, thoải mái di chuyển. (không rõ cái này nên scrape thành cái gì)
- 2433: Hẻm 2,5 m đường 2/4 (scrape thành 2,5m)
- 246:  Đường vào rộng rãi, dễ dàng di chuyển bằng ô tô.
- 247: Nhà vị trí đẹp, ngõ rộng, xe máy quay đầu thoải mái, mặt tiền rộng thoáng
- 248: Đương oto thông 2 mặt tiền trước sau. (scrape thành 0 vì câu này nhưng thực chất nó là hẻm xe tải ấy)
- 253: Đường trước nhà ô tô tránh, (mà scrape thành 4)
- 265: Đường 10 mét ở đường Bà Hạt - Phường 6 - Quận 10.\n* Vị trí gần Chợ Nguyễn Tri Phương, thông ra đường 3 Tháng 2. (scrape thành 2 do cái này nhưng thực tình cũng không biết nên là cái gì)
- 273: NHÀ 2 MẶT TIỀN - HẺM ÔTÔ LÝ THUYẾT (scrape thành 0)
- 275: Phố Đường Thành gần Hàng Điếu, Hàng Da, Bát Đàn, Hàng Nón, Hàng Gà.\n- Phố kinh doanh vô cùng sầm uất, nhiều nhà hàng nổi tiếng, ngân hàng, shop thời trang, khách sạn, homestay.\n- Thiết kế:\n- Tầng 1: Sân, bếp, phòng khách, phòng ngủ.\n- Tầng 2: 2 ngủ khép kín.\n- Tầng 3: Phòng kho, sân phơi.\n\nPháp lý: Sổ đỏ chính chủ sẵn sàng giao dịch.\nGiá 23.5 tỷ có thương lượng
- 283: ngõ rộng sạch sẽ, xe máy quay đầu thoải mái, mặt tiền rộng, kinh doanh tốt (scrape thành 0 do cái bà cứ thêm Liên Nhà Phố)
- 288: Siêu phẩm góc 2 mặt tiền
- 290: Nhà vị trí đẹp mặt tiền rộng kinh doanh thuận tiện, đường trước nhà rộng ô tô tránh nhau dừng đỗ thoải mái. (scrape thành 0 nhưng không có gì đảm bảo chắc chắn)
- 303: Ngõ vào rộng 5m (scrape thành 0)
- 304:  ngõ kinh doanh phù hợp mọi loại hình, ngõ to rộng 2 ô tô tránh thẳng tắp gần UBND quận Đống Đa (scrape thành 0)
- 310:Trục đường Vành Đai 2.5 giao thông thuận tiện...\n+ Ngõ đẹp ô tô tránh. (scrape thành 0)
- 312
- 313: ngõ trước nhà 
- 320: hẻm rộng 6m²
- 328: Bán nhà đẹp Phú Diễn - 6 tầng thang máy, ô tô đỗ cửa Giá chỉ 9,1 tỷ\n\nDiện tích 38m x 6 tầng, mặt tiền 4m
- 331: Gia chủ đi định cư bán gấp nhà hẻm xe hơi 449 Sư Vạn Hạnh, P12, Q10
- 342: Vị trí gần Toà Án Nhân Dân Quận Gò Vấp tầm 100m.
- 345
- 351
- 353: Hẻm trước nhà 6m 2 xe hơi tránh nhau thoải mái
- 359: Hẻm nội bộ cực rộng 12m
- 360:  ra sân bay chỉ 5 phút vào Quận 1, Quận 3 không tới 10 phút.
- 363: Bán nhà riêng, ô tô vào nhà, mặt tiền 6,5m, không bị án ngữ, gần cầu Nhật Tân, view Hồ Tây, sông Hồng, các khu đô thị VIP Ciputra, Sunshine, TTTM lotte, trường cấp 1,2,3 quận Tây Hồ, chợ dân sinh buôn bán sầm uất, các bệnh viện lớn, đi sân bay 15 phút... Sổ đỏ chính chủ, pháp lý rõ ràng.	
- 371
- 372: Ngõ rộng ô tô đỗ cửa không cần né
- 377: đường rộng 6m đường Thống Nhất Phường 15 Quận Gò Vấp (scrape thành 6m)
- 378: Đường vào 3m, xe hơi ra vào thoải mái, rất thuận tiện cho việc di chuyển.
- 380: Cần bán nhà hẻm xe hơi
- 382
- 385: Mặt tiền hẻm 7m thông thoáng, xe hơi quay đầu thoải mái.
- 387
- 389
- 390
- 391
- 398: Vị trí : mặt tiền hẻm 8m Lê Văn Quới thông đường hương lộ 2 kế bên Ngã Tư 4 Xã f Bình Trị Đông A quận Bình Tân
- 399: Ngõ vào ô tô thoải mái, thuận tiện cho việc di chuyển
- 407:  Hẻm nhựa 10m có vỉa hè, mặt tiền hẻm kinh doanh toà nhà văn phòng, kế Etown Cộng Hoà, ga T3
- 408: Ngõ to, cực thoáng, cách bãi gửi ô tô chỉ 50m
- 410: Đường trước nhà ô tô tránh nhau thoải mái, đường nhựa ô bàn cờ.
- 412: cần bán nhà cấp 4 đường nội bộ có sẵn lề đường 12m
- 415: Bán nhà mặt ngõ lớn kinh doanh 
- 417
- 418
- 420
- 421
- 423
- 424
- 427
- 429
- 431
- 433: Ngõ vào rộng 8m, thuận tiện cho xe hơi. Mặt tiền rộng 8m, phù hợp cho kinh doanh hoặc cho thuê.
- 435: Cần bán nhà HXH
- 436: Nhà vị trí đẹp, ngõ rộng sạch sẽ, xe máy quay đầu thoải mái, mặt tiền rộng, kinh doanh tốt
- 437
- 439
- 440: mặt tiền 4m, tọa lạc tại Phố Trần Quốc Hoàn
- 442: Vị trí cực đẹp ngõ thông gần ngã tư thái hà - hoàng cầu ngõ ô tô thông tứ tung, gần đường tàu điện vài bước chân ra mặt phố 2 chiều.
- 443

In [58]:
# df.dropna(subset='Đường vào', axis=0, inplace=True)
df['digit_road'] = df['Đường vào'].apply(lambda x: x.split()[0].replace(',', '.').strip() if x is not None else None)
df['digit_road'] = df['digit_road'].astype(float)

# Test Area (Diện tích đất)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27491 entries, 0 to 27490
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27491 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27491 non-null  object 
 2   Xã/Phường/Thị trấn                    27469 non-null  object 
 3   Đường phố                             23802 non-null  object 
 4   Chi tiết                              27491 non-null  object 
 5   url                                   27491 non-null  object 
 6   Tình trạng giao dịch                  27491 non-null  object 
 7   Thời điểm giao dịch/rao bán           27479 non-null  object 
 8   Thông tin liên hệ                     0 non-null      object 
 9   Giá rao bán/giao dịch                 24953 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27491 non-null  object 
 11  Số tầng công tr

In [190]:
def area(row):
    if row['other_info'] != {}:
        return row['other_info'].get('Diện tích')
    return row['main_info'][1].get('value')

df['area'] = df.apply(area, axis=1)
print(df.shape[0])
df = df[~(df['area'] == '')]
print(df.shape[0])
# df = df[df['area'].str.contains('m²')]
# df['digit_area'] = df['area'].apply(lambda x: float(x.split()[0].replace('.', '').replace(',','.')))

27491
27489


# Test number of floors (Số tầng công trình)

In [ ]:
import pandas as pd
import json

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)

listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
df = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')

df['other_info'] = df['other_info'].apply(json.loads)
df['main_info'] = df['main_info'].apply(json.loads)

In [65]:
df['yes_floor'] = df['other_info'].apply(lambda x: x.get('Số tầng') if x.get('Số tầng') else None)
df['yes_floor'] = df['yes_floor'].apply(lambda x: int(x.split()[0]) if x is not None else None)
print(df[~df['yes_floor'].isna()].shape[0])
no_floor_in_other_info = df[df['yes_floor'].isna()]

16464


In [2]:
df.shape[0]

18536

In [4]:
import re
import numpy as np
# from rapidfuzz import fuzz

def check_additional_floor(value):
    additional_floor = ['sân thượng', 'sân thương', ' st ', 'trệt', 'trêt', 'tret', 'tum', 'hầm', 'hâm', 'gác lửng', 'gác mái', 'lửng', 'lững', 'lừng']
    result = 0
    for word in additional_floor:
        if word in value:
        # tokens = value.split()
        # for token in tokens:
        #     if fuzz.ratio(token, word) > 70:
            if word == 'sân thượng' or word == 'sân thương':
                additional_floor.remove(' st ')
            if word == ' st ':
                additional_floor.remove('sân thượng')
            print(f'Additional Floor word detected: {word}')
            result += 1
    return result

def clean_num_floor(row):
    print(f'Cleaning for row {row["index"]}')
    floor_keywords = ['tầng', 'lầu', 'tấm', 'mê']
    word_to_num = {
            "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
            "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
            "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
            "mười lăm": 15, "mười sáu": 16
        }
    # reuse word_to_num from before
    num_words_pattern = "|".join(sorted(word_to_num.keys(), key=lambda x: -len(x)))
    # ------- TH1: Thông tin đã có sẵn ở other_info ------
    if row['other_info'] != {} and row['other_info'].get('Số tầng'):
        return int(row['other_info'].get('Số tầng').split()[0])
    # ------ TH2: Nhà cũ/nhà cấp 4 ở title/description ------
    if pd.notna(row['title']):
        lower_title = row['title'].lower()
        old_house = re.search(pattern=r'nhà (?:\w+\s){0,5}cũ', string=lower_title)
        if old_house:
            return 0
        cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_title)
        if cap4:
            return 1
    if pd.notna(row['description']):
        try:
            lower_des = row['description'].lower()
            old_house = re.search(pattern=r'nhà (?:\w+\s){0,5}cũ', string=lower_des)
            if old_house:
                return 0
            cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_des)
            if cap4:
                return 1
        except:
            print(row['description'] == np.nan)
    # ------ TH3: Xét số tầng ------
    # Tống số tầng trong title
    if pd.notna(row['title']):
        add_key = check_additional_floor(lower_title)
        for keyword in floor_keywords:
            if keyword in lower_title:
                num_floor = re.search(pattern=rf'(\d|{num_words_pattern})\s*{keyword}', string=lower_title)
                if num_floor:
                    print(f'Extracted floor in title: {num_floor.group(1)}')
                    print(f'Additional value: {add_key}')
                    if num_floor.group(1).isdigit():
                        possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*{keyword}', string=lower_title)
                        if possible_float:
                            print(f'Found possible float: {possible_float.group(1)}')
                            return float(possible_float.group(1).replace(',','.')) + add_key
                        return int(num_floor.group(1)) + add_key
                    return word_to_num[num_floor.group(1)] + add_key
                    # elif num_floor.group(1) in word_to_num.keys():
                    #     index_needed.append(row['index'])
                    #     return word_to_num[num_floor.group(1)] + add_key
    if pd.notna(row['description']):
        add_key = check_additional_floor(lower_des)
        # Trong trường hợp nêu rõ tầng 1, tầng 2,... thì max sẽ là tổng số tầng
        total_pattern = re.findall(pattern=r'(?:tầng|lầu|tấm|mê)\s* ([\d\w]+):', string=lower_des)
        if total_pattern:
            print(f"Extracted total floor in description: {total_pattern}")
            total_floor_num = []
            for digit in total_pattern:
                if digit.isdigit():
                    total_floor_num.append(int(digit))
                elif digit in word_to_num.keys():
                    total_floor_num.append(word_to_num[digit])
            if total_floor_num:
                return max(total_floor_num)
        
        # Trong trường hợp liệt kê ra cả lố tầng thì là cộng tổng vào
        separate_pattern = re.search(pattern=rf'(\d|{num_words_pattern})\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
        if separate_pattern:
            print(f'Extracted floor that needs to be sum up: {separate_pattern.group(1)}')
            print(f'Additional value: {add_key}')
            if separate_pattern.group(1).isdigit():
                possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
                if possible_float:
                    print(f'Found possible float: {possible_float.group(1)}')
                    return float(possible_float.group(1).replace(',','.')) + add_key
                return int(separate_pattern.group(1)) + add_key
            return word_to_num[separate_pattern.group(1)] + add_key
    return 'Không ghi rõ'
    # elif row['description'] is not None:
    #     lower_des = row['description'].lower()
    #     old_house = re.search(pattern=rf'nhà [\w+\s]{0-5}cũ', string=lower_des)
    #     if old_house:
    #         return 0
    #     cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_des)
    #     if cap4:
    #         return 1
        
    #     add_key = check_additional_floor(lower_des)

df['index'] = df.index
df['floor_extracted_from'] = ''
df['floor'] = df.apply(clean_num_floor, axis = 1)

Cleaning for row 0
Cleaning for row 1
Cleaning for row 2
Cleaning for row 3
Cleaning for row 4
Cleaning for row 5
Cleaning for row 6
Cleaning for row 7
Cleaning for row 8
Cleaning for row 9
Cleaning for row 10
Cleaning for row 11
Cleaning for row 12
Cleaning for row 13
Cleaning for row 14
Cleaning for row 15
Cleaning for row 16
Cleaning for row 17
Cleaning for row 18
Cleaning for row 19
Cleaning for row 20
Cleaning for row 21
Cleaning for row 22
Cleaning for row 23
Cleaning for row 24
Cleaning for row 25
Cleaning for row 26
Cleaning for row 27
Extracted floor in title: 6
Additional value: 0
Cleaning for row 28
Cleaning for row 29
Cleaning for row 30
Cleaning for row 31
Cleaning for row 32
Cleaning for row 33
Cleaning for row 34
Cleaning for row 35
Cleaning for row 36
Cleaning for row 37
Cleaning for row 38
Cleaning for row 39
Cleaning for row 40
Additional Floor word detected: hầm
Extracted floor in title: 6
Additional value: 1
Cleaning for row 41
Cleaning for row 42
Cleaning for row 4

## Code extract num floor ở dưới này

In [5]:
# Code không lỗi 
import re
import numpy as np
from rapidfuzz import fuzz

# def check_additional_floor(value):
#     additional_floor = ['sân thượng', 'sân thương', ' st ', 'trệt', 'trêt', 'tret', 'tum', 'hầm', 'hâm', 'gác lửng', 'gác mái', 'lửng', 'lững', 'lừng']
#     result = 0
#     for word in additional_floor:
#         if word in value:
#         # tokens = value.split()
#         # for token in tokens:
#         #     if fuzz.ratio(token, word) > 70:
#             if word == 'sân thượng' or word == 'sân thương':
#                 additional_floor.remove(' st ')
#             if word == ' st ':
#                 additional_floor.remove('sân thượng')
#             print(f'Additional Floor word detected: {word}')
#             result += 1
#     return result

def new_check_additional_floor(string, additional_floor):
    result = 0
    for word in additional_floor:
        if word in string:
            result += 1
    search_st = re.search(pattern=r'(\Wst\W)', string=string)
    if search_st:
        result += 1
    search_gl = re.search(pattern=r'gác lửng|gác lững|ghác lửng|ghác lững', string=string)
    if search_gl:
        result -= 1
    return result 

def is_float(num):
    try:
        float(num)
        return True
    except:
        return False

def extract_separate(lower_value, floor_keywords):
    value_list = lower_value.split()
    forbidden_pattern = r'giấy phép xây dựng|giấy phép xây|phép xây dựng|gpxd|có thể xây|cải tạo|được phép xây'
    additional_floor = ['sân thượng', 'sân thương', 'trêt', 'trệt', 'tret', 'tum', 'hầm', 'hâm', 'gác', 'gac', 'lửng', 'lững', 'lừng']
    if 'trệt' in floor_keywords:
        additional_floor = ['sân thượng', 'sân thương',  'tum', 'hầm', 'hâm', 'gác', 'gac', 'lửng', 'lững', 'lừng']
    word_to_num = {
            "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
            "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
            "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
            "mười lăm": 15, "mười sáu": 16
        }
    for keyword in floor_keywords: # Check cho từng chiếc keyword
        if keyword in lower_value: # Nếu trong string có một trong những chiếc keyword
            i = 0
            while i < len(value_list):
                # for value in value_list: # Tìm trong list string mà đã được tách ra sẵn
            #     if keyword in value: # Nếu chiếc keyword đã tìm thấy ban nãy là của từ này
                    # word_index = value_list.index(value) # Lấy index của từ
                if keyword in value_list[i]: # Tìm index của chiếc từ keyword floor 
                    # Trong trường hợp mà nó bị dính chữ vào với nhau
                    extracted_floor = value_list[i].replace(keyword, '').replace(',','.')
                    if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()): 
                        word_lower = i - 4 if i - 4 >= 0 else 0
                        word_upper = i + 4 if i + 4 < len(value_list) else len(value_list) - 1
                        search_range = ' '.join(value_list[word_lower:word_upper + 1]) # Tìm trong khoảng 5 từ trước - sau của từ
                        forbidden_word = re.search(pattern=forbidden_pattern, string=search_range)
                        if forbidden_word: # Nếu xuất hiện forbidden word
                            i += 1
                        else:
                            if 'hiện trạng' in ' '.join(value_list[word_lower:i]): # Nếu hiện trạng 3 tầng --> return luôn
                                if is_float(extracted_floor):
                                    return abs(float(extracted_floor))
                                if extracted_floor in word_to_num.keys():
                                    return word_to_num[extracted_floor]
                            else:
                                if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()):
                                    add_key = new_check_additional_floor(search_range, additional_floor)
                                    if is_float(extracted_floor):
                                        return abs(float(extracted_floor)) + add_key
                                    return word_to_num[extracted_floor] + add_key
                                else:
                                    add_key = new_check_additional_floor(search_range, additional_floor)
                                    if add_key > 0:
                                        return add_key + 1
                                    else:
                                        i += 1
                    # Nếu có thể extract vị trí ở đằng trước keyword đã cho
                    elif i - 1 >= 0:
                        extracted_floor = value_list[i - 1].replace(',', '.')
                        word_lower = i - 4 if i - 4 >= 0 else 0
                        word_upper = i + 4 if i + 4 < len(value_list) else len(value_list) - 1
                        search_range = ' '.join(value_list[word_lower:word_upper + 1]) # Tìm trong khoảng 5 từ trước - sau của từ
                        forbidden_word = re.search(pattern=forbidden_pattern, string=search_range)
                        if forbidden_word: # Nếu xuất hiện forbidden word
                            i += 1
                        else:
                            if 'hiện trạng' in ' '.join(value_list[word_lower:i]): # Nếu hiện trạng 3 tầng --> return luôn
                                if is_float(extracted_floor):
                                    return abs(float(extracted_floor))
                                elif extracted_floor in word_to_num.keys():
                                    return word_to_num[extracted_floor]
                                else:
                                    i += 1
                            else:
                                if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()):
                                    add_key = new_check_additional_floor(search_range, additional_floor)
                                    if is_float(extracted_floor):
                                        return abs(float(extracted_floor)) + add_key
                                    return word_to_num[extracted_floor] + add_key
                                else:
                                    add_key = new_check_additional_floor(search_range, additional_floor)
                                    if add_key > 0:
                                        return add_key + 1
                                    else:
                                        i += 1
                    else:
                        i += 1
                else:
                    i += 1
    
    return None


def clean_num_floor(row):
    floor_keywords = ['tầng', 'lầu', 'tấm', 'mê']
    word_to_num = {
            "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
            "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
            "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
            "mười lăm": 15, "mười sáu": 16
        }
    # ------- TH1: Thông tin đã có sẵn ở other_info ------
    if row['other_info'] != {} and row['other_info'].get('Số tầng'):
        return int(row['other_info'].get('Số tầng').split()[0])
    # ------ TH2: Nhà cũ/nhà cấp 4 ở title/description ------
    # Xét của description trước do description thường được viết đầy đủ hơn
    if pd.notna(row['description']):
        lower_des = row['description'].lower().replace('+', ' ').replace('x',' ').replace('*', ' ')
        old_house = re.search(pattern=r'nhà (?:\w+\s*){0,5}cũ|bán(?:\s+\S\s*){0,5}đất', string=lower_des)
        if old_house:
            return 0
        cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt|nhà nát', string=lower_des)
        if cap4:
            if cap4.group(0) == 'nhà trệt':
                extract_result = extract_separate(lower_des, floor_keywords)
                if extract_result:
                    return extract_result
            return 1
    if pd.notna(row['title']):
        lower_title = row['title'].lower().replace('+', ' ').replace('x',' ').replace('*', ' ')
        old_house = re.search(pattern=r'nhà (?:\w+\s*){0,5}cũ|bán(?:\s+\S\s*){0,5}đất', string=lower_title)
        if old_house:
            return 0
        cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt|nhà nát', string=lower_title)
        if cap4:
            if cap4.group(0) == 'nhà trệt':
                extract_result = extract_separate(lower_title, floor_keywords)
                if extract_result:
                    return extract_result
            return 1
    # ------ TH3: Xét tổng số tầng ------
        extract_result = extract_separate(lower_title, floor_keywords)
        if extract_result:
            return extract_result
    # Có lẽ với trường hợp này, nếu có add_key thì skip xuống extract description cho đủ, nếu description không na hoặc kệ luôn
    # if pd.notna(row['title']):
    #     add_key = check_additional_floor(lower_title)
    #     for keyword in floor_keywords:
    #         if keyword in lower_title:
    #             num_floor = re.search(pattern=rf'(\d|{num_words_pattern})\s*{keyword}', string=lower_title)
    #             if num_floor:
    #                 print(f'Extracted floor in title: {num_floor.group(1)}')
    #                 print(f'Additional value: {add_key}')
    #                 if num_floor.group(1).isdigit():
    #                     possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*{keyword}', string=lower_title)
    #                     if possible_float:
    #                         print(f'Found possible float: {possible_float.group(1)}')
    #                         return float(possible_float.group(1).replace(',','.')) + add_key, 'title'
    #                     return int(num_floor.group(1)) + add_key, 'title'
    #                 return word_to_num[num_floor.group(1)] + add_key, 'title'
    #                 # elif num_floor.group(1) in word_to_num.keys():
    #                 #     index_needed.append(row['index'])
    #                 #     return word_to_num[num_floor.group(1)] + add_key
    if pd.notna(row['description']):
        # add_key = check_additional_floor(lower_des)
        # Trong trường hợp nêu rõ tầng 1, tầng 2,... thì max sẽ là tổng số tầng
        total_pattern = re.findall(pattern=r'(?:tầng|lầu|tấm|mê)\s* ([\d\w]+):', string=lower_des)
        if total_pattern:
            total_floor_num = []
            for digit in total_pattern:
                if is_float(digit):
                    total_floor_num.append(abs(float(digit)))
                elif digit in word_to_num.keys():
                    total_floor_num.append(word_to_num[digit])
            if total_floor_num:
                return max(total_floor_num)
    #------TH4: Xét số tầng mà có miêu tả cấu trúc cụ thể (Kiểu như trệt 2 lầu)------
        else:
            extract_result = extract_separate(lower_des, floor_keywords)
            if extract_result:
                return extract_result
            else:
                extract_result = extract_separate(lower_des, ['trệt', 'trêt', 'tret'])
                if extract_result:
                    return extract_result  
                extract_result = extract_separate(lower_title, ['trệt', 'trêt', 'tret'])     
                if extract_result:
                    return extract_result       
    # if pd.notna(row['description']):
    #     extract_result = extract_separate(lower_des)
    #     if extract_result:
    #         return extract_result, 'description'
    # if pd.notna(row['title']):
    #     extract_result = extract_separate(lower_title)
    #     if extract_result:
    #         return extract_result, 'title'
        # # Trong trường hợp liệt kê ra cả lố tầng thì là cộng tổng vào
        # separate_pattern = re.search(pattern=rf'(\d|{num_words_pattern})\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
        # if separate_pattern:
        #     print(f'Extracted floor that needs to be sum up: {separate_pattern.group(1)}')
        #     print(f'Additional value: {add_key}')
        #     if separate_pattern.group(1).isdigit():
        #         possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
        #         if possible_float:
        #             print(f'Found possible float: {possible_float.group(1)}')
        #             return float(possible_float.group(1).replace(',','.')) + add_key, 'description'
        #         return int(separate_pattern.group(1)) + add_key, 'description'
        #     return word_to_num[separate_pattern.group(1)] + add_key, 'description'
    return 1
    # elif row['description'] is not None:
    #     lower_des = row['description'].lower()
    #     old_house = re.search(pattern=rf'nhà [\w+\s]{0-5}cũ', string=lower_des)
    #     if old_house:
    #         return 0
    #     cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_des)
    #     if cap4:
    #         return 1
        
    #     add_key = check_additional_floor(lower_des)

df['index'] = df.index
df['Số tầng công trình'] = df.apply(clean_num_floor, axis = 1, result_type='expand')

In [6]:
# Code không có hiện trạng
import re
import numpy as np
from rapidfuzz import fuzz

# def check_additional_floor(value):
#     additional_floor = ['sân thượng', 'sân thương', ' st ', 'trệt', 'trêt', 'tret', 'tum', 'hầm', 'hâm', 'gác lửng', 'gác mái', 'lửng', 'lững', 'lừng']
#     result = 0
#     for word in additional_floor:
#         if word in value:
#         # tokens = value.split()
#         # for token in tokens:
#         #     if fuzz.ratio(token, word) > 70:
#             if word == 'sân thượng' or word == 'sân thương':
#                 additional_floor.remove(' st ')
#             if word == ' st ':
#                 additional_floor.remove('sân thượng')
#             print(f'Additional Floor word detected: {word}')
#             result += 1
#     return result

def new_check_additional_floor(string, additional_floor):
    result = 0
    for word in additional_floor:
        if word in string:
            result += 1
    search_st = re.search(pattern=r'(\Wst\W)', string=string)
    if search_st:
        result += 1
    search_gl = re.search(pattern=r'gác lửng|gác lững|ghác lửng|ghác lững', string=string)
    if search_gl:
        result -= 1
    return result 

def is_float(num):
    try:
        float(num)
        return True
    except:
        return False

def extract_separate(lower_value, floor_keywords):
    value_list = lower_value.split()
    forbidden_pattern = r'giấy phép xây dựng|giấy phép xây|phép xây dựng|gpxd|có thể xây|cải tạo|được phép xây'
    additional_floor = ['sân thượng', 'sân thương', 'trêt', 'trệt', 'tret', 'tum', 'hầm', 'hâm', 'gác', 'gac', 'lửng', 'lững', 'lừng']
    if 'trệt' in floor_keywords:
        additional_floor = ['sân thượng', 'sân thương',  'tum', 'hầm', 'hâm', 'gác', 'gac', 'lửng', 'lững', 'lừng']
    word_to_num = {
            "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
            "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
            "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
            "mười lăm": 15, "mười sáu": 16
        }
    for keyword in floor_keywords: # Check cho từng chiếc keyword
        if keyword in lower_value: # Nếu trong string có một trong những chiếc keyword
            i = 0
            while i < len(value_list):
                # for value in value_list: # Tìm trong list string mà đã được tách ra sẵn
            #     if keyword in value: # Nếu chiếc keyword đã tìm thấy ban nãy là của từ này
                    # word_index = value_list.index(value) # Lấy index của từ
                if keyword in value_list[i]: # Tìm index của chiếc từ keyword floor 
                    # Trong trường hợp mà nó bị dính chữ vào với nhau
                    extracted_floor = value_list[i].replace(keyword, '').replace(',','.')
                    if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()): 
                        word_lower = i - 4 if i - 4 >= 0 else 0
                        word_upper = i + 4 if i + 4 < len(value_list) else len(value_list) - 1
                        search_range = ' '.join(value_list[word_lower:word_upper + 1]) # Tìm trong khoảng 5 từ trước - sau của từ
                        forbidden_word = re.search(pattern=forbidden_pattern, string=search_range)
                        if forbidden_word: # Nếu xuất hiện forbidden word
                            i += 1
                        else:
                            # if 'hiện trạng' in ' '.join(value_list[word_lower:i]): # Nếu hiện trạng 3 tầng --> return luôn
                            #     if is_float(extracted_floor):
                            #         return abs(float(extracted_floor))
                            #     if extracted_floor in word_to_num.keys():
                            #         return word_to_num[extracted_floor]
                            # else:
                            if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()):
                                add_key = new_check_additional_floor(search_range, additional_floor)
                                if is_float(extracted_floor):
                                    return abs(float(extracted_floor)) + add_key
                                return word_to_num[extracted_floor] + add_key
                            else:
                                add_key = new_check_additional_floor(search_range, additional_floor)
                                if add_key > 0:
                                    return add_key + 1
                                else:
                                    i += 1
                    # Nếu có thể extract vị trí ở đằng trước keyword đã cho
                    elif i - 1 >= 0:
                        extracted_floor = value_list[i - 1].replace(',', '.')
                        word_lower = i - 4 if i - 4 >= 0 else 0
                        word_upper = i + 4 if i + 4 < len(value_list) else len(value_list) - 1
                        search_range = ' '.join(value_list[word_lower:word_upper + 1]) # Tìm trong khoảng 5 từ trước - sau của từ
                        forbidden_word = re.search(pattern=forbidden_pattern, string=search_range)
                        if forbidden_word: # Nếu xuất hiện forbidden word
                            i += 1
                        else:
                            # if 'hiện trạng' in ' '.join(value_list[word_lower:i]): # Nếu hiện trạng 3 tầng --> return luôn
                            #     if is_float(extracted_floor):
                            #         return abs(float(extracted_floor))
                            #     elif extracted_floor in word_to_num.keys():
                            #         return word_to_num[extracted_floor]
                            #     else:
                            #         i += 1
                            # else:
                            if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()):
                                add_key = new_check_additional_floor(search_range, additional_floor)
                                if is_float(extracted_floor):
                                    return abs(float(extracted_floor)) + add_key
                                return word_to_num[extracted_floor] + add_key
                            else:
                                add_key = new_check_additional_floor(search_range, additional_floor)
                                if add_key > 0:
                                    return add_key + 1
                                else:
                                    i += 1
                    else:
                        i += 1
                else:
                    i += 1
    
    return None


def clean_num_floor(row):
    floor_keywords = ['tầng', 'lầu', 'tấm', 'mê']
    word_to_num = {
            "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
            "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
            "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
            "mười lăm": 15, "mười sáu": 16
        }
    # ------- TH1: Thông tin đã có sẵn ở other_info ------
    if row['other_info'] != {} and row['other_info'].get('Số tầng'):
        return int(row['other_info'].get('Số tầng').split()[0])
    # ------ TH2: Nhà cũ/nhà cấp 4 ở title/description ------
    # Xét của description trước do description thường được viết đầy đủ hơn
    if pd.notna(row['description']):
        lower_des = row['description'].lower().replace('+', ' ').replace('x',' ').replace('*', ' ')
        old_house = re.search(pattern=r'nhà (?:\w+\s*){0,5}cũ|bán(?:\s+\S\s*){0,5}đất', string=lower_des)
        if old_house:
            return 0
        cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt|nhà nát', string=lower_des)
        if cap4:
            if cap4.group(0) == 'nhà trệt':
                extract_result = extract_separate(lower_des, floor_keywords)
                if extract_result:
                    return extract_result
            return 1
    if pd.notna(row['title']):
        lower_title = row['title'].lower().replace('+', ' ').replace('x',' ').replace('*', ' ')
        old_house = re.search(pattern=r'nhà (?:\w+\s*){0,5}cũ|bán(?:\s+\S\s*){0,5}đất', string=lower_title)
        if old_house:
            return 0
        cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt|nhà nát', string=lower_title)
        if cap4:
            if cap4.group(0) == 'nhà trệt':
                extract_result = extract_separate(lower_title, floor_keywords)
                if extract_result:
                    return extract_result
            return 1
    # ------ TH3: Xét tổng số tầng ------
        extract_result = extract_separate(lower_title, floor_keywords)
        if extract_result:
            return extract_result
    # Có lẽ với trường hợp này, nếu có add_key thì skip xuống extract description cho đủ, nếu description không na hoặc kệ luôn
    # if pd.notna(row['title']):
    #     add_key = check_additional_floor(lower_title)
    #     for keyword in floor_keywords:
    #         if keyword in lower_title:
    #             num_floor = re.search(pattern=rf'(\d|{num_words_pattern})\s*{keyword}', string=lower_title)
    #             if num_floor:
    #                 print(f'Extracted floor in title: {num_floor.group(1)}')
    #                 print(f'Additional value: {add_key}')
    #                 if num_floor.group(1).isdigit():
    #                     possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*{keyword}', string=lower_title)
    #                     if possible_float:
    #                         print(f'Found possible float: {possible_float.group(1)}')
    #                         return float(possible_float.group(1).replace(',','.')) + add_key, 'title'
    #                     return int(num_floor.group(1)) + add_key, 'title'
    #                 return word_to_num[num_floor.group(1)] + add_key, 'title'
    #                 # elif num_floor.group(1) in word_to_num.keys():
    #                 #     index_needed.append(row['index'])
    #                 #     return word_to_num[num_floor.group(1)] + add_key
    if pd.notna(row['description']):
        # add_key = check_additional_floor(lower_des)
        # Trong trường hợp nêu rõ tầng 1, tầng 2,... thì max sẽ là tổng số tầng
        total_pattern = re.findall(pattern=r'(?:tầng|lầu|tấm|mê)\s* ([\d\w]+):', string=lower_des)
        if total_pattern:
            total_floor_num = []
            for digit in total_pattern:
                if is_float(digit):
                    total_floor_num.append(abs(float(digit)))
                elif digit in word_to_num.keys():
                    total_floor_num.append(word_to_num[digit])
            if total_floor_num:
                return max(total_floor_num)
    #------TH4: Xét số tầng mà có miêu tả cấu trúc cụ thể (Kiểu như trệt 2 lầu)------
        else:
            extract_result = extract_separate(lower_des, floor_keywords)
            if extract_result:
                return extract_result
            else:
                extract_result = extract_separate(lower_des, ['trệt', 'trêt', 'tret'])
                if extract_result:
                    return extract_result  
                extract_result = extract_separate(lower_title, ['trệt', 'trêt', 'tret'])     
                if extract_result:
                    return extract_result       
    # if pd.notna(row['description']):
    #     extract_result = extract_separate(lower_des)
    #     if extract_result:
    #         return extract_result, 'description'
    # if pd.notna(row['title']):
    #     extract_result = extract_separate(lower_title)
    #     if extract_result:
    #         return extract_result, 'title'
        # # Trong trường hợp liệt kê ra cả lố tầng thì là cộng tổng vào
        # separate_pattern = re.search(pattern=rf'(\d|{num_words_pattern})\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
        # if separate_pattern:
        #     print(f'Extracted floor that needs to be sum up: {separate_pattern.group(1)}')
        #     print(f'Additional value: {add_key}')
        #     if separate_pattern.group(1).isdigit():
        #         possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
        #         if possible_float:
        #             print(f'Found possible float: {possible_float.group(1)}')
        #             return float(possible_float.group(1).replace(',','.')) + add_key, 'description'
        #         return int(separate_pattern.group(1)) + add_key, 'description'
        #     return word_to_num[separate_pattern.group(1)] + add_key, 'description'
    return 1
    # elif row['description'] is not None:
    #     lower_des = row['description'].lower()
    #     old_house = re.search(pattern=rf'nhà [\w+\s]{0-5}cũ', string=lower_des)
    #     if old_house:
    #         return 0
    #     cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_des)
    #     if cap4:
    #         return 1
        
    #     add_key = check_additional_floor(lower_des)

df['index'] = df.index
df['New Số tầng công trình'] = df.apply(clean_num_floor, axis = 1, result_type='expand')

In [17]:
string = 'kết cấu tòa nhà: trệt 3 lầu sân thượng thoáng đãng'
pattern = r'(?:\S+\s+){0,5}(\d+(?:[.,]\d+)*|\w+)\s*lầu (?:\S+\s+){0,5}'
yes = re.search(pattern, string)
if yes:
    print(yes.group(1))

3


## New Code using Regex

In [5]:
# TRY NEW ONE
import re
import numpy as np
from rapidfuzz import fuzz

def new_check_additional_floor(string):
    additional_floor = ['sân thượng', 'sân thương',  'tum', 'hầm', 'hâm', 'gác', 'gac', 'lửng', 'lững', 'lừng']
    result = 0
    for word in additional_floor:
        if word in string:
            result += 1
    search_st = re.search(pattern=r'(\Wst\W)', string=string)
    if search_st:
        result += 1
    search_tret = re.search(pattern=r'(?:trệt|trêt|tret)\s*(?:\S+\s+){0,3}(?:tầng|lầu|tấm|mê)', string=string)
    if search_tret:
        result += 1
    search_gl = re.search(pattern=r'gác lửng|gác lững|ghác lửng|ghác lững', string=string)
    if search_gl:
        result -= 1
    return result 

def is_float(num):
    try:
        float(num)
        return True
    except:
        return False
    
def extract_separate(lower_value, floor_keywords):
    # value_list = lower_value.split()
    forbidden_pattern = 'giấy phép xây dựng|giấy phép xây|phép xây dựng|gpxd|có thể xây|cải tạo|được phép xây'
    # additional_floor = ['sân thượng', 'sân thương', 'trêt', 'trệt', 'tret', 'tum', 'hầm', 'hâm', 'gác', 'gac', 'lửng', 'lững', 'lừng']
    # if 'trệt' in floor_keywords:
    
    word_to_num = {
            "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
            "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
            "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
            "mười lăm": 15, "mười sáu": 16
        }
    floor_key = '|'.join(key for key in floor_keywords)
    found_floor_key = re.findall(rf'{floor_key}', lower_value)
    for key in found_floor_key:
        forbidden_word = re.search(rf'{forbidden_pattern}\s*(?:\S+\s*){{0,2}}{key}', lower_value)
        if forbidden_word:
            continue
        extracted_floor = re.search(rf'(?:\S+\s*){{0,5}}(\d+(?:[.,]\d+)*|\w+)\s*{key}\s*(?:\S+\s*){{0,5}}', lower_value)
        if extracted_floor:
            extracted_floor_num = extracted_floor.group(1).replace(',', '.')
            add_key = new_check_additional_floor(extracted_floor.group(0))
            if is_float(extracted_floor_num) or (extracted_floor_num in word_to_num.keys()):
                if is_float(extracted_floor_num):
                    return abs(float(extracted_floor_num)) + add_key
                return word_to_num[extracted_floor_num] + add_key
            else:
                if add_key > 0:
                    return add_key + 1 # Nếu trường hợp trệt lầu sân thượng thì phải cộng 1 cho cái lầu nữa
    return None
    
def new_clean_num_floor(row):
    floor_keywords = ['tầng', 'lầu', 'tấm', 'mê']
    word_to_num = {
            "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
            "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
            "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
            "mười lăm": 15, "mười sáu": 16
        }
    # ------- TH1: Thông tin đã có sẵn ở other_info ------
    if row['other_info'] != {} and row['other_info'].get('Số tầng'):
        return int(row['other_info'].get('Số tầng').split()[0])
    # ------ TH2: Nhà cũ/nhà cấp 4 ở title/description ------
    # Xét của description trước do description thường được viết đầy đủ hơn
    if pd.notna(row['description']):
        lower_des = row['description'].lower().replace('+', ' ').replace('x',' ').replace('*', ' ')
        old_house = re.search(pattern=r'nhà (?:\w+\s*){0,5}cũ|bán(?:\s+\S\s*){0,5}đất', string=lower_des)
        if old_house:
            return 0
        cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt|nhà nát', string=lower_des)
        if cap4:
            if cap4.group(0) == 'nhà trệt':
                extract_result = extract_separate(lower_des, floor_keywords)
                if extract_result:
                    return extract_result
            return 1
    if pd.notna(row['title']):
        lower_title = row['title'].lower().replace('+', ' ').replace('x',' ').replace('*', ' ')
        old_house = re.search(pattern=r'nhà (?:\w+\s*){0,5}cũ|bán(?:\s+\S\s*){0,5}đất', string=lower_title)
        if old_house:
            return 0
        cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt|nhà nát', string=lower_title)
        if cap4:
            if cap4.group(0) == 'nhà trệt':
                extract_result = extract_separate(lower_des, floor_keywords)
                if extract_result:
                    return extract_result
            return 1
    # ------ TH3: Xét tổng số tầng ------
        extract_result = extract_separate(lower_title, floor_keywords)
        if extract_result:
            return extract_result
    if pd.notna(row['description']):
        total_pattern = re.findall(pattern=r'(?:tầng|lầu|tấm|mê)\s* ([\d\w]+):', string=lower_des)
        if total_pattern:
            total_floor_num = []
            for digit in total_pattern:
                if is_float(digit):
                    total_floor_num.append(abs(float(digit)))
                elif digit in word_to_num.keys():
                    total_floor_num.append(word_to_num[digit])
            if total_floor_num:
                return max(total_floor_num)
    #------TH4: Xét số tầng mà có miêu tả cấu trúc cụ thể (Kiểu như trệt 2 lầu)------
        else:
            extract_result = extract_separate(lower_des, floor_keywords)
            if extract_result:
                return extract_result
            else:
                extract_result = extract_separate(lower_des, ['trệt', 'trêt', 'tret'])
                if extract_result:
                    return extract_result  
                extract_result = extract_separate(lower_title, ['trệt', 'trêt', 'tret'])     
                if extract_result:
                    return extract_result       
    return 1

df['New Số tầng công trình'] = df.apply(new_clean_num_floor, axis = 1)
            

Sai
- 64: Scrape từ 5 thành 1
- 452: 1 trệt 1 gác (scrape từ 2 thành 1)
- 1487: trệt+lửng mà scrape thành 1
- 2122: 1 Trêt + 3lầu mà scrape thành 3
- 2294: trệt lửng (scrape thành 1)
- 3658: 1trệt 2lầu (scrape thành 2)
- 3670: 1 trệt 1 lửng (scrape thành 1)
- 3768: 1 trệt, 1 gác lửng (scrape thành 1)
- 3923: Trệt - Lầu (scrape thành 1)
- 6412:  trệt lửng (scrape thành 1)
- 6521: DT: 51m2_3tầng (scrape thành 1)
- 6562: Chính chủ muốn bán căn hộ tầng 2 trên khối nhà Pháp cổ 2 tầng (scrape thành 1)
- 6583: kết cấu 1 trệt, lầu, 1 lửng (scrape thành 1)
- 68431: 1 trệt 1 lầu (scrape thành 1)

Đúng
- 148: Scrape 8 thành 9
- 356: scrape từ 2 (chả hiểu lấy đâu ra) thành 1
- 2224: Phù hợp xây cao tầng 6,7 tầng (scrape thành 1)
- 5212: Kết cấu 4 tầng gồm 1 trệt , 2 lầu , sân thượng (scrape từ 5 thành 4)

In [7]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

df[df['Số tầng công trình'] != df['New Số tầng công trình']][['description', 'title', 'other_info', 'Số tầng công trình', 'New Số tầng công trình']]

,description,title,other_info,Số tầng công trình,New Số tầng công trình
64,"Bán CHDV 12 phòng full nội thất gần Hiệp Thành City Quận 12, giá 9.5 tỷ tl\n\n-DT: 5x20m full thổ, hẻm 7m thông\n- 1 hầm 1 trệt 3 lầu có thang máy\n-Tổng 12 phòng full nội thất đều có bancol và máy giặt riêng.\n-Thang máy lên tới tầng thượng. Có thể cải tạo thêm 3 phòng nữa, đã chừa đường nước và điện\n-Trang bị đầy đủ hệ thống PCCC, thang máy, wifi, camera, cửa cuốn ...\n\nGiá bán: 9.5 tỷ có thương lượng","Bán CHDV 12 phòng full nội thất gần Hiệp Thành City Quận 12, giá 9.5 tỷ tl","{'Mức giá': '9,5 tỷ', 'Diện tích': '100 m²'}",5.0,1.0
148,"+ Bán Nhà Phân Lô Phố Ngọc Thụy , Khu Vip Nhà Giàu , Kinh Doanh Mọi ngành nghề\n+ Thiết kế 70m² * 8 Tầng Thang máy Kinh Doanh + Pen House , Nhà có Hầm để 4 Xe Oto Tiện lợi , Mặt Phố Phân Lô Rộng 12M Vỉa hè Thông Thoáng Nhộn Nhịp\n+ Sổ Đỏ Phân Lô Vuông Đẹp , Nhà Cho Thuê Dòng Tiền Lớn , Giá Có Thương Lượng","PHÂN LÔ NGỌC THỤY - DT 70M2 * 8 TẦNG THANG MÁY , CÓ HẦM KINH DOANH , MẶT PHỐ 12 M VỈA HÈ , 29 TỶ","{'Mức giá': 'Thỏa thuận', 'Diện tích': '70 m²', 'Pháp lý': 'Sổ đỏ/ Sổ hồng'}",8.0,9.0
356,"- Vị tri: Trung tâm quận Tân Phú, đầy đủ tiện ích, tiện qua Aeon mall Tân Phú, tiện qua Q.TB, Q10\n- Nhà mới vào ở ngay, 5PN, có phòng ngủ tầng trệt tiện cho người cao tuổi ở\n- Hẻm nhựa 6m để ô tô ngày đêm, khu an ninh, yên tĩnh, nhà cao tầng liền kề.\n- Pháp lý chuẩn, hoàn công đủ, công chứng ngay.","Bán nhà Gò Dầu hẻm nhựa 6m,72m2(4x18),5PN, sổ vuông,hoàn công đủ, giá nhỉnh 9tỷ(còn thương lượng)","{'Mức giá': '9 tỷ', 'Diện tích': '70 m²'}",2.0,1.0
2122,"*** giá 7ty9 TL\nBán nhà Nguyễn Trãi, Phường Nguyễn Cư Trinh, Quận 1\n- DTCN: 30.6m². DT 3.5x10m\n- Kết cấu: 1 Trêt + 3lầu + 5PN+ 4WC.\n- nhà đang cho thuê. Nhà ngay cạnh Khu căn hộ dịch vụ cao cấp Lancaster Legacy. Cách phố đi bộ Bùi Viện 500m.\n*** Giá: 7.9tỷ TL\nLH/ Điệp\n0903 070 ***","Bán nhà Nguyễn Trãi, Phường Nguyễn Cư Trinh, Quận 1","{'Mức giá': '7,9 tỷ', 'Diện tích': '30,6 m²', 'Mặt tiền': '3,5 m'}",4.0,3.0
2224,"Bán nhà phố Bùi Huy Bích.\nDT 145m, mặt tiền rộng 7,6m.\nĐường rộng, oto tránh giao thông thuận tiện, phù hợp xây cao tầng 6, 7 tầng.\nVị tri: Trung tâm quận Hoàng mai, gần bến xe Nước Ngầm, bến xe Giáp Bát, gần khu Linh Đàm.\nSổ đỏ chính chủ.\nLH:\n0989 604 ***\n.","Bán nhà phố Bùi Huy Bích 145m2, oto tránh phân lô, LH 0989 604 ***","{'Mức giá': '40 tỷ', 'Diện tích': '145 m²', 'Mặt tiền': '7,6 m', 'Pháp lý': 'Sổ đỏ'}",7.0,1.0
2294,"Bán nhà 3 mặt hẻm xe tải 7m và hẻm oto 4,5m và hẻm mặt sau nhà. 50m ra đường lê đức thọ. Đầu đường cf highlands, gym, xung quanh đầy đủ tiện nghi\nDT đất 41m², trệt lửng. Cực kỳ thoáng\nHệ số xây dựng 4.5\nHẻm sau nhà tương lai quy hoạch đường 20m thông\nNgân hàng BIDV định giá 5t, cho vay tới 3,6t. Cần tiền bán nhanh 4,9t chốt\nHoa hồng 1%, nhờ các ace môi giới chạy dùm\n\nPháp lý đầy đủ, sổ đỏ sẵn sàng.\nHướng cửa chính Đông Nam, giúp mang lại tài lộc cho gia chủ.\n\nLiên hệ ngay để biết thêm chi tiết: trang,\n0372 331 ***\n.","3 mặt tiền, hẻm xe tải, vừa ở vừa kinh doanh","{'Mức giá': '4,9 tỷ', 'Diện tích': '41 m²', 'Số phòng ngủ': '2 phòng', 'Số phòng tắm, vệ sinh': '1 phòng', 'Hướng nhà': 'Đông - Nam', 'Mặt tiền': '3,5 m', 'Đường vào': '7 m', 'Pháp lý': 'Sổ đỏ/ Sổ hồng', 'Nội thất': 'Không nội thất'}",2.0,1.0
3658,"Bán Nhà 1trệt 2lầu, hẻm 1693 Nguyễn Duy Trinh PTrường Thạnh, Q9\nDT: 51m² ( ngang 4.4m ) Giá 4.290tỷ\n- hoàn công đầy đủ\n- sân, pk, bếp, 3pn, 3wc\n- Hẻm oto\nLh\n0931 850 ***\nMr Nhân","Bán Nhà 1trệt 2lầu, hẻm 1693 Nguyễn Duy Trinh PTrường Thạnh, Q9","{'Mức giá': '4,29 tỷ', 'Diện tích': '51 m²', 'Số phòng ngủ': '3 phòng', 'Số phòng tắm, vệ sinh': '3 phòng', 'Pháp lý': 'Sổ đỏ/ Sổ hồng'}",3.0,2.0
3923,"*hẻm xe 7 chổ vào thoải mái ngủ trong nhà\n*100% sổ vuông\n* VỊ TRÍ : phường 3 gò vấp\n+ gần trung tâm gò vấp sầm uất\n+ gần trường học các cấp\n+ khu thương mại bật nhất, ăn uống vui chơi mọi lứa tuổi\n+ gần bệnh viện quân y 175 , trụ sở công an, chợ Tân Sơn\n* THIẾT KẾ : BTCT CHẮC CHẮN\n+ Nhà hiện tại Trệt - Lầu,

In [ ]:
# import re
# import numpy as np
# from rapidfuzz import fuzz

# def check_additional_floor(value):
#     additional_floor = ['sân thượng', 'sân thương', ' st ', 'trệt', 'trêt', 'tret', 'tum', 'hầm', 'hâm', 'gác lửng', 'gác mái', 'lửng', 'lững', 'lừng']
#     result = 0
#     for word in additional_floor:
#         if word in value:
#         # tokens = value.split()
#         # for token in tokens:
#         #     if fuzz.ratio(token, word) > 70:
#             if word == 'sân thượng' or word == 'sân thương':
#                 additional_floor.remove(' st ')
#             if word == ' st ':
#                 additional_floor.remove('sân thượng')
#             print(f'Additional Floor word detected: {word}')
#             result += 1
#     return result

# def new_check_additional_floor(string, additional_floor):
#     print(f'Search string: {string}')
#     result = 0
#     for word in additional_floor:
#         if word in string:
#             result += 1
#     search_st = re.search(pattern=r'(\Wst\W)', string=string)
#     if search_st:
#         result += 1
#     return result 

# def is_float(num):
#     try:
#         float(num)
#         return True
#     except:
#         return False

# def extract_separate(lower_value, floor_keywords):
#     value_list = lower_value.split()
#     forbidden_pattern = r'giấy phép xây dựng|giấy phép xây|phép xây dựng|gpxd|có thể xây|cải tạo|được phép xây'
#     additional_floor = ['sân thượng', 'sân thương', 'trêt', 'trệt', 'tret', 'tum', 'hầm', 'hâm', 'gác mái', 'lửng', 'lững', 'lừng']
#     if 'trệt' in floor_keywords:
#         additional_floor = ['sân thượng', 'sân thương',  'tum', 'hầm', 'hâm', 'gác mái', 'lửng', 'lững', 'lừng']
#     word_to_num = {
#             "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
#             "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
#             "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
#             "mười lăm": 15, "mười sáu": 16
#         }
#     for keyword in floor_keywords: # Check cho từng chiếc keyword
#         if keyword in lower_value: # Nếu trong string có một trong những chiếc keyword
#             i = 0
#             while i < len(value_list):
#                 print(f"Index {i} for keyword: {keyword}")
#                 print(f'Word: {value_list[i]}')
#                 # for value in value_list: # Tìm trong list string mà đã được tách ra sẵn
#             #     if keyword in value: # Nếu chiếc keyword đã tìm thấy ban nãy là của từ này
#                     # word_index = value_list.index(value) # Lấy index của từ
#                 if keyword in value_list[i]: # Tìm index của chiếc từ keyword floor 
#                     # Trong trường hợp mà nó bị dính chữ vào với nhau
#                     if value_list[i].endswith(keyword):
#                         print(value_list[i])
#                         extracted_floor = value_list[i].replace(keyword, '').replace(',','.')
#                         if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()): 
#                             word_lower = i - 4 if i - 4 >= 0 else 0
#                             word_upper = i + 4 if i + 4 < len(value_list) else len(value_list) - 1
#                             search_range = ' '.join(value_list[word_lower:word_upper + 1]) # Tìm trong khoảng 5 từ trước - sau của từ
#                             forbidden_word = re.search(pattern=forbidden_pattern, string=search_range)
#                             if forbidden_word: # Nếu xuất hiện forbidden word
#                                 i += 1
#                                 continue
#                             else:
#                                 if 'hiện trạng' in ' '.join(value_list[word_lower:i]): # Nếu hiện trạng 3 tầng --> return luôn
#                                     if is_float(extracted_floor):
#                                         print(f'Extracted value: {abs(float(extracted_floor))}')
#                                         return abs(float(extracted_floor))
#                                     if extracted_floor in word_to_num.keys():
#                                         print(f'Extracted value: {word_to_num[extracted_floor]}')
#                                         return word_to_num[extracted_floor]
#                                 else:
#                                     if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()):
#                                         print(f'Extracted Floor: {extracted_floor}')
#                                         add_key = new_check_additional_floor(search_range, additional_floor)
#                                         if is_float(extracted_floor):
#                                             print(f'Extracted value with add key: {abs(float(extracted_floor)) + add_key}')
#                                             return abs(float(extracted_floor)) + add_key
#                                         print(f'Extracted Floor: {extracted_floor}')
#                                         print(f'Extracted value with add key: {word_to_num[extracted_floor] + add_key}')
#                                         return word_to_num[extracted_floor] + add_key
#                                     else:
#                                         add_key = new_check_additional_floor(search_range, additional_floor)
#                                         if add_key > 0:
#                                             print(f'Extracted value using add key only: {add_key + 1}')
#                                             return add_key + 1
#                     else:
#                         # Nếu có thể extract vị trí ở đằng trước keyword đã cho
#                         if i - 1 >= 0:
#                             extracted_floor = value_list[i - 1].replace(',', '.')
#                             word_lower = i - 4 if i - 4 >= 0 else 0
#                             word_upper = i + 4 if i + 4 < len(value_list) else len(value_list) - 1
#                             search_range = ' '.join(value_list[word_lower:word_upper + 1]) # Tìm trong khoảng 5 từ trước - sau của từ
#                             forbidden_word = re.search(pattern=forbidden_pattern, string=search_range)
#                             if forbidden_word: # Nếu xuất hiện forbidden word
#                                 i += 1
#                                 continue
#                             else:
#                                 if 'hiện trạng' in ' '.join(value_list[word_lower:i]): # Nếu hiện trạng 3 tầng --> return luôn
#                                     if is_float(extracted_floor):
#                                         print(f'Extracted value: {abs(float(extracted_floor))}')
#                                         return abs(float(extracted_floor))
#                                     elif extracted_floor in word_to_num.keys():
#                                         print(f'Extracted value: {word_to_num[extracted_floor]}')
#                                         return word_to_num[extracted_floor]
#                                     # else:
#                                     #     i += 1
#                                 else:
#                                     if is_float(extracted_floor) or (extracted_floor in word_to_num.keys()):
#                                         add_key = new_check_additional_floor(search_range, additional_floor)
#                                         if is_float(extracted_floor):
#                                             print(f'Extracted value with add key: {abs(float(extracted_floor)) + add_key}')
#                                             return abs(float(extracted_floor)) + add_key
#                                         print(f'Extracted value with add key: {word_to_num[extracted_floor] + add_key}')
#                                         return word_to_num[extracted_floor] + add_key
#                                     else:
#                                         add_key = new_check_additional_floor(search_range, additional_floor)
#                                         if add_key > 0:
#                                             return add_key + 1
#                                         # else:
#                                         #     i += 1
#                         # else:
#                         #     i += 1
#                 # else:
#                 #     i += 1
#                 i += 1
    
#     return None


# def clean_num_floor(row):
#     print(f'Cleaning for row {row["index"]}')
#     floor_keywords = ['tầng', 'lầu', 'tấm', 'mê']
#     word_to_num = {
#             "một": 1, "hai": 2, "ba": 3, "bốn": 4, "năm": 5, "sáu": 6,
#             "bảy": 7, "bẩy": 7, "tám": 8, "chín": 9, "mười": 10,
#             "mười một": 11, "mười hai": 12, "mười ba": 13, "mười bốn": 14,
#             "mười lăm": 15, "mười sáu": 16
#         }
#     # ------- TH1: Thông tin đã có sẵn ở other_info ------
#     if row['other_info'] != {} and row['other_info'].get('Số tầng'):
#         return int(row['other_info'].get('Số tầng').split()[0]), 'other_info'
#     # ------ TH2: Nhà cũ/nhà cấp 4 ở title/description ------
#     # Xét của description trước do description thường được viết đầy đủ hơn
#     if pd.notna(row['description']):
#         lower_des = row['description'].lower().replace('+', ' ').replace('x',' ').replace('*', ' ')
#         old_house = re.search(pattern=r'nhà (?:\w+\s*){0,5}cũ', string=lower_des)
#         if old_house:
#             return 0, 'description'
#         cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_des)
#         if cap4:
#             if cap4.group(0) == 'nhà trệt':
#                 extract_result = extract_separate(lower_des, floor_keywords)
#                 if extract_result:
#                     return extract_result, 'description'
#             return 1, 'description'
#     if pd.notna(row['title']):
#         lower_title = row['title'].lower().replace('+', ' ').replace('x',' ').replace('*', ' ')
#         old_house = re.search(pattern=r'nhà (?:\w+\s*){0,5}cũ', string=lower_title)
#         if old_house:
#             return 0, 'title'
#         cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_title)
#         if cap4:
#             if cap4.group(0) == 'nhà trệt':
#                 extract_result = extract_separate(lower_title, floor_keywords)
#                 if extract_result:
#                     return extract_result, 'title'
#             return 1, 'title'
#     # ------ TH3: Xét tổng số tầng ------
#         extract_result = extract_separate(lower_title, floor_keywords)
#         if extract_result:
#             return extract_result, 'title'
#     # Có lẽ với trường hợp này, nếu có add_key thì skip xuống extract description cho đủ, nếu description không na hoặc kệ luôn
#     # if pd.notna(row['title']):
#     #     add_key = check_additional_floor(lower_title)
#     #     for keyword in floor_keywords:
#     #         if keyword in lower_title:
#     #             num_floor = re.search(pattern=rf'(\d|{num_words_pattern})\s*{keyword}', string=lower_title)
#     #             if num_floor:
#     #                 print(f'Extracted floor in title: {num_floor.group(1)}')
#     #                 print(f'Additional value: {add_key}')
#     #                 if num_floor.group(1).isdigit():
#     #                     possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*{keyword}', string=lower_title)
#     #                     if possible_float:
#     #                         print(f'Found possible float: {possible_float.group(1)}')
#     #                         return float(possible_float.group(1).replace(',','.')) + add_key, 'title'
#     #                     return int(num_floor.group(1)) + add_key, 'title'
#     #                 return word_to_num[num_floor.group(1)] + add_key, 'title'
#     #                 # elif num_floor.group(1) in word_to_num.keys():
#     #                 #     index_needed.append(row['index'])
#     #                 #     return word_to_num[num_floor.group(1)] + add_key
#     if pd.notna(row['description']):
#         # add_key = check_additional_floor(lower_des)
#         # Trong trường hợp nêu rõ tầng 1, tầng 2,... thì max sẽ là tổng số tầng
#         total_pattern = re.findall(pattern=r'(?:tầng|lầu|tấm|mê)\s* ([\d\w]+):', string=lower_des)
#         if total_pattern:
#             print(f"Extracted total floor in description: {total_pattern}")
#             total_floor_num = []
#             for digit in total_pattern:
#                 if is_float(digit):
#                     total_floor_num.append(abs(float(digit)))
#                 elif digit in word_to_num.keys():
#                     total_floor_num.append(word_to_num[digit])
#             if total_floor_num:
#                 return max(total_floor_num), 'description'
#     #------TH4: Xét số tầng mà có miêu tả cấu trúc cụ thể (Kiểu như trệt 2 lầu)------
#         else:
#             extract_result = extract_separate(lower_des, floor_keywords)
#             if extract_result:
#                 return extract_result, 'description'
#             else:
#                 extract_result = extract_separate(lower_des, ['trệt', 'trêt', 'tret'])
#                 if extract_result:
#                     return extract_result, 'description trệt'   
#                 extract_result = extract_separate(lower_title, ['trệt', 'trêt', 'tret'])     
#                 if extract_result:
#                     return extract_result, 'title trệt'        
#     # if pd.notna(row['description']):
#     #     extract_result = extract_separate(lower_des)
#     #     if extract_result:
#     #         return extract_result, 'description'
#     # if pd.notna(row['title']):
#     #     extract_result = extract_separate(lower_title)
#     #     if extract_result:
#     #         return extract_result, 'title'
#         # # Trong trường hợp liệt kê ra cả lố tầng thì là cộng tổng vào
#         # separate_pattern = re.search(pattern=rf'(\d|{num_words_pattern})\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
#         # if separate_pattern:
#         #     print(f'Extracted floor that needs to be sum up: {separate_pattern.group(1)}')
#         #     print(f'Additional value: {add_key}')
#         #     if separate_pattern.group(1).isdigit():
#         #         possible_float = re.search(pattern=rf'(\d+[.,]\d+)\s*(?:tầng|lầu|tấm|mê)', string=lower_des)
#         #         if possible_float:
#         #             print(f'Found possible float: {possible_float.group(1)}')
#         #             return float(possible_float.group(1).replace(',','.')) + add_key, 'description'
#         #         return int(separate_pattern.group(1)) + add_key, 'description'
#         #     return word_to_num[separate_pattern.group(1)] + add_key, 'description'
#     return 1, 'NaN values'
#     # elif row['description'] is not None:
#     #     lower_des = row['description'].lower()
#     #     old_house = re.search(pattern=rf'nhà [\w+\s]{0-5}cũ', string=lower_des)
#     #     if old_house:
#     #         return 0
#     #     cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4|nc4|nhà trệt', string=lower_des)
#     #     if cap4:
#     #         return 1
        
#     #     add_key = check_additional_floor(lower_des)

# df['index'] = df.index
# df[['floor', 'floor_extracted_from']] = df.apply(clean_num_floor, axis = 1, result_type='expand')

In [71]:
clean_num_floor(df.iloc[412])

1

In [11]:
print(f"Title new: {df[df['floor_extracted_from'] == 'title'].shape[0]}")
print(f"Description new: {df[df['floor_extracted_from'] == 'description'].shape[0]}")
print(f"Title trệt new: {df[df['floor_extracted_from'] == 'title trệt'].shape[0]}")
print(f"Description trệt new: {df[df['floor_extracted_from'] == 'description trệt'].shape[0]}")
print(f"NaN values new: {df[df['floor_extracted_from'] == 'NaN values'].shape[0]}")

Title new: 280
Description new: 1112
Title trệt new: 0
Description trệt new: 16
NaN values new: 9376


# Test Shape (Hình dạng)

In [28]:
import pandas as pd
import json

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)

listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
df = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')

df['other_info'] = df['other_info'].apply(json.loads)
df['main_info'] = df['main_info'].apply(json.loads)

In [55]:
df['Hình dạng'].value_counts()

Hình dạng
Chữ nhật                     25123
Nở hậu                        2065
Chữ L                           21
Thóp hậu                        10
Tam giác                         8
Chữ T                            5
Chữ U                            3
Đa giác từ 5 cạnh, méo mó        2
Chữ nhật vát góc                 1
Name: count, dtype: int64

In [ ]:
from src.config import NEGATION_PATTERNS, SHAPE_KEYWORDS

def extract_shape(row):
    def is_negated(text: str, kw: str) -> bool:
            for pattern in NEGATION_PATTERNS:
                if re.search(pattern.format(re.escape(kw)), text):
                    return True
            return False

    text = f"{row.get('title', '')} {row.get('description', '')}".lower()

    for shape, kws in SHAPE_KEYWORDS.items():
        for kw in kws:
            if re.search(rf"\b{re.escape(kw)}\b", text) and not is_negated(text, kw):
                return shape, kw

    return "Chữ nhật", 'NaN'

In [70]:
from src.config import NEGATION_PATTERNS, SHAPE_KEYWORDS
from sentence_transformers import SentenceTransformer, util

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Precompute embeddings for your keywords
shape_embeddings = {}
for shape, kws in SHAPE_KEYWORDS.items():
    shape_embeddings[shape] = [(kw, model.encode(kw, convert_to_tensor=True)) for kw in kws]

def new_extract_shape(row, threshold=0.7):
    def is_negated(text: str, kw: str) -> bool:
        for pattern in NEGATION_PATTERNS:
            if re.search(pattern.format(re.escape(kw)), text):
                return True
        return False

    text = f"{row.get('title', '')} {row.get('description', '')}".lower()
    words = re.findall(r'\w+', text)

    for word in words:
        word_vec = model.encode(word, convert_to_tensor=True)
        for shape, kw_embs in shape_embeddings.items():
            for kw, kw_vec in kw_embs:
                sim = util.cos_sim(word_vec, kw_vec).item()
                if sim >= threshold and not is_negated(text, kw):
                    return (shape, word)

    return ('Chữ nhật', 'NaN')

In [ ]:
df[['shape', 'shape_keyword']] = df.apply(extract_shape, axis = 1, result_type='expand')

In [ ]:
df[['new_shape', 'new_shape_keyword']] = df.apply(new_extract_shape, axis = 1, result_type='expand')

#  Test Chất lượng còn lại 

In [1]:
import pandas as pd
import json
import numpy as np

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)

listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
df = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')

df['other_info'] = df['other_info'].apply(json.loads)
df['main_info'] = df['main_info'].apply(json.loads)

In [30]:
from rapidfuzz import fuzz
import re

QUALITY_LEVELS = [
    # Priority 1: Structure has essentially no value (0%)
    (0.0, [
        'tặng nhà', 'bán đất tặng nhà', 'chỉ tính tiền đất',
        'đất nền', 'nhà tạm', 'chủ yếu lấy đất', 'tặng nhà'
        'có nhà nhưng không đáng giá', 'không tính giá trị nhà',
        'nhà cấp 4 cũ', 'nhà xuống cấp', 'giá trị đất là chính',
        'bán đất'
    ]),
    # Priority 2: Old or needs significant repair (50%)
    (0.5, [
        'nhà cũ', 'nhà nát', 'cần sửa chữa', 'tiện xây mới',
        'xây lâu năm', 'xuống cấp', 'cũ nhưng ở tạm được',
        'cũ kỹ', 'nhiều năm chưa sửa', 'cần cải tạo',
        'nền móng yếu', 'sắp sập', 'cần xây lại', 
        'không có giá trị sử dụng',
    ]),
    # Priority 3: Good, well-maintained condition (85%)
    (0.85, [
        'nhà đẹp', 'còn mới', 'giữ gìn', 'full nội thất', 'thiết kế hiện đại',
        'nhà sạch sẽ', 'ở ngay', 'nhà gọn gàng', 'nội thất cao cấp',
        'không cần sửa', 'đẹp như hình', 'vào ở liền',
        'nội thất đầy đủ', 'tiện nghi', 'nhà không lỗi phong thủy',
        'còn bảo hành', 'nhà chất lượng tốt',
    ]),
    # Priority 4: Brand-new condition (100%)
    (1.0, [
        'mới xây', 'mới hoàn thiện', 'mới 100%', 'nhà mới keng',
        'vừa xây xong', 'mới bàn giao', 'mới nhận nhà', 'nhà rất mới',
        'chưa ở lần nào', 'nhà mới tinh', 'nhà mới toanh',
        'nhà xây mới', 'vừa hoàn thiện', 'còn thơm mùi sơn',
        'mới hoàn công', 'nhà xây kiên cố', 'đảm bảo kết cấu mới',
    ]),   
]

DEFAULT_QUALITY = 0.75

def estimate_remaining_quality(row):
    text = f"{row.get('title', '')} {row.get('description', '')}".lower()
    result = {}
    # result_qual = 0.75
    # result_ratio = 0
    for quality_val, keywords in QUALITY_LEVELS:
        for kw in keywords:
            pattern = kw.replace(' ', '(?:\s*\w+\s*){0,2} ')
            pattern = pattern.strip()
            pattern = '\W' + pattern + '\W'
            qual = re.search(pattern, text)
            if qual:
                ratio = fuzz.ratio(kw, qual.group(0))
                if quality_val == 0 or quality_val == 1:
                    ratio += 3
                result[quality_val] = [qual.group(0), ratio]
                # if ratio >= result_ratio:
                #     result_ratio = ratio
                #     result_qual = round(quality_val, 2)
                #     # result_qual.append(round(quality_val, 2))
                # return round(quality_val, 2), qual.group(0)
    # return round(DEFAULT_QUALITY, 2), 'None'
    # if 0 in result_qual:
    #     result_qual = 0
    # if 1 in result_qual:
    #     result_qual = 1
    # else:
    #     result_qual = max(result_qual)
    if result:
        # Sort by ratio first, then by quality value
        best_quality, (match, score) = max(result.items(), key=lambda x: (x[1][1], x[0]))
        return best_quality, result 
    else:
        return DEFAULT_QUALITY, result

    # return result_qual, result

In [92]:
string = df.loc[5]['description']
print(string)
pattern = 'full(?:\s*\S+\s){0,2} nội(?:\s*\S+\s){0,2}'
print(f"{re.search(pattern, string).group(0) + 'hhh'}")

Toà nhà hẻm 8m Hoàng Hoa Thám, P.13 Tân Bình, hiện đang cho thuê khoán làm căn hộ dịch vụ 100tr/tháng. Thích hợp mua giữ tiền..
Diện tích: Đất 130m², vuông vắn. Sàn sử dụng 650m².
Hiện trạng: Nhà 5 tầng. Có thang máy, gồm 32 phòng full nội thất, cho thuê khoán là 100tr/tháng. Tự khai thác 150tr/tháng.
Hẻm rộng 8m thông, ngay sát nhà ga T3. Khu vực nhiều văn phòng, trung tâm thương mại.
Giá 19.2 tỷ thương lượng.
LH:
0903 992 ***
. MTG.
full nội thất, cho hhh


In [155]:
estimate_remaining_quality(df.loc[69])

(1.0,
 {0.85: [' tiện nghi', 94.73684210526316],
  1.0: [' mới xây', 96.33333333333333]})

In [31]:
df['quality keyword'] = ''
df[['new quality', 'quality keyword']] = df.apply(estimate_remaining_quality, axis=1, result_type = 'expand')

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

df[df['new quality'] != df['Chất lượng còn lại']][['new quality', 'Chất lượng còn lại', 'description', 'title']].iloc[:10]

In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

print(f'Quality level 0: {df[df["Chất lượng còn lại"] == 0.0].shape[0]}')
print(f'Quality level 0.5: {df[df["Chất lượng còn lại"] == 0.5].shape[0]}')
print(f'Quality level 0.75: {df[df["Chất lượng còn lại"] == 0.75].shape[0]}')
print(f'Quality level 0.85: {df[df["Chất lượng còn lại"] == 0.85].shape[0]}')
print(f'Quality level 1: {df[df["Chất lượng còn lại"] == 1].shape[0]}')

Quality level 0: 8774
Quality level 0.5: 482
Quality level 0.75: 9595
Quality level 0.85: 7049
Quality level 1: 1348


In [28]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

print(f'Quality level 0: {df[df["new quality"] == 0.0].shape[0]}')
print(f'Quality level 0.5: {df[df["new quality"] == 0.5].shape[0]}')
print(f'Quality level 0.75: {df[df["new quality"] == 0.75].shape[0]}')
print(f'Quality level 0.85: {df[df["new quality"] == 0.85].shape[0]}')
print(f'Quality level 1: {df[df["new quality"] == 1].shape[0]}')

Quality level 0: 5150
Quality level 0.5: 497
Quality level 0.75: 12152
Quality level 0.85: 8014
Quality level 1: 1435


# Test Đơn giá xây dựng

Các loại đơn giá xây dựng:
- Nhà cấp 4: 4,000,000
- Nhà 1 tầng bê tông cốt thép: 6,275,876
- Nhà từ 2 tầng, bê tông cốt thép, có hầm: 9,504,604
- Nhà từ 2 tầng, bê tông cốt thép, không hầm: 8,221,171
- Nhà biệt thự: 10,510,920
- Nhà biệt thự có hầm: 12,848,184

In [1]:
import pandas as pd
import json
import numpy as np

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls', 'description'], axis=1, inplace=True)

listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
df = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')

df['other_info'] = df['other_info'].apply(json.loads)
df['main_info'] = df['main_info'].apply(json.loads)

In [88]:
yo = 'hầm + trệt + 3 lầu sân thượng akl dn hs'
print(yo)
yo = yo.replace('+', ' ')
pattern = '(?:\S+\s+){0,5}lầu(?:\s*\S+){0,5}'
string = yo.lower()
print(re.findall(pattern, string))

hầm + trệt + 3 lầu sân thượng akl dn hs
['hầm   trệt   3 lầu sân thượng akl dn hs']


In [51]:
import re

def check_ham(text):
    # text = text.replace(',', ' ').replace('+', ' ')
    string = re.findall(pattern=r'(?:(?!được xây|giấy phép xây dựng|gpxd|cải tạo)\b\w+\b\W+){1,7}hầm(?:(?!\schui)\W+\b\w+\b){1,7}', string=text)
    if string:
        for substr in string:
            ham = re.search(pattern=r'tầng|lầu|tấm|mê|\d+|xe|kết cấu|kc|ô tô|thang máy|trệt|lửng', string=substr)
            if ham:
                return True
    return False
    
def extract_construction_cost(row):
    title = row['title']
    title_lower = title.lower()
    if pd.notna(row['description']):
        des_lower = row['description'].lower()
        text = f'{title_lower} {des_lower}'
    else:
        text = title_lower
        des_lower = 'none'
    text = text.replace(',',' ').replace('+',' ').replace('\n',' ').replace('*', ' ')
    text = ' '.join(text.split())
    des_lower = des_lower.replace(',',' ').replace('+',' ').replace('\n',' ').replace('*', ' ')
    des_lower = ' '.join(des_lower.split())
    if 'nhà trệt' in text:
        if not re.search(r'nhà trệt\s*(?:\S+\s+){0,2}(?:\d*\s*)(?:tầng|lầu|tấm|mê)', text):
            return 4000000
    cap4 = re.search(pattern = r'nhà cấp 4|nhà c4|cấp 4\W|nc4|nhà trệt|nhà nát', string=text)
    if cap4:
        return 4000000 #4,000,000
    if row['Số tầng công trình'] == 1:
        return 6275876
    
    ham = check_ham(text)
    if re.search(r'biệt thự', title_lower) or re.search(r'villa\W', title_lower):
    # if 'biệt thự' in title_lower or 'villa' in title_lower:
        if ham:
            return 12848184
        return 10510920
    if des_lower != 'none' and (re.search(r'biệt thự', des_lower) or re.search(r'villa\W', des_lower)):
        villa_pattern = [
            r'(?:thiết kế|xây)*\s*(?:\S+\s+){0,5} (?:phong cách|kiểu|dạng|kiến trúc|cấu trúc)\s*(?:\S+\s+){0,2} (?:biệt thự|villa\W)', #Các nhà có cấu trúc villa
            r'bán (?:(?!mua|xây)\S+\s+){0,3}(?:biệt thự|villa\W)', # Bán biệt thự
            r'(?:biệt thự|villa\W)\s*(?:\S+\s+){0,3}\d+\s*tầng' # Biệt thự bao nhiêu tầng
        ]
        not_villa_pattern = [
            r'(?:đối diện|nằm|sát|cạnh|ngay|liền kề|hàng xóm|xung quanh|gần|view|nhiều)\s*(?:\S+\s+){0,5}\s*(?:biệt thự|villa)', # Bên cạnh là khu villa
            r'(?:làm|xây|cải tạo)\s*(\S+\s+){0,4}(?:biệt thự|villa)', # Có thể xây thành biệt thự
            r'(?:nhà|phố|mặt tiền|tòa nhà|building|chuyên|kinh doanh|chdv|căn hộ dịch vụ|kdt|kđt|khu đô thị|(?:\+84|0)\s*(?:\d\s*){3,6}(?:\d\s*){0,3}|văn phòng|cao ốc|nhà cao tầng)(?:\s+\S+){0,5} (?:biệt thự|villa)', # Tránh giới thiệu về cò
            r'(?:biệt thự|villa)(?:\s+\S+){0,5} (?:nhà|phố|mặt tiền|tòa nhà|building|chuyên|kinh doanh|chdv|căn hộ dịch vụ|kdt|kđt|khu đô thị|(?:\+84|0)\s*(?:\d\s*){3,6}(?:\d\s*){0,3}|văn phòng|cao ốc|nhà cao tầng)', # Tránh giới thiệu về cò
            r'(?:mua|xây) (\S+\s+){0,3}(?:biệt thự|villa)', # Loại các trường bán để chuyển qua mua hoặc xây biệt thự
            r'(?:như|khu|toàn)\s*(?:\S+\s+){0,1}(?:biệt thự|villa)', # Các trường hợp đẹp như biệt thự, khu biệt thự
            r'(?:ra|chuyển)\s*(?:\S+\s+){0,2}(?:biệt thự|villa)' # Chuyển ra để ở khu villa
        ]
        for pattern in villa_pattern:
            if re.search(pattern, des_lower):
                if ham:
                    return 12848184 # Biệt thự có hầm
                return 10510920 # Biệt thự không hầm
        for pattern in not_villa_pattern:
            if re.search(pattern, des_lower):
                if row['Số tầng công trình'] == 2:
                    if ham:
                        return 6275876 # Nhà 1 tầng 1 hầm
                    else:
                        return 8221171 # Nhà 2 tầng không hầm
                if row['Số tầng công trình'] < 2:
                    if ham:
                        return 6275876 # ví dụ như nhà 1.5 thì 0.5 đó chính là tầng hầm 
                    return 8221171 # Nhà 2 tầng không hầm
                if ham:
                    return 9504604 # Nhà hơn 2 tầng, có hầm
                return 8221171 # Nhà hơn 2 tầng, không hầm
        if ham:
            return 12848184 # Biệt thự có hầm
        return 10510920 # Biệt thự không hầm
    if row['Số tầng công trình'] == 2:
        if ham:
            return 6275876 # Nhà 1 tầng 1 hầm
        else:
            return 8221171 # Nhà 2 tầng không hầm
    if row['Số tầng công trình'] < 2:
        if ham:
            return 6275876 # ví dụ như nhà 1.5 thì 0.5 đó chính là tầng hầm 
        return 8221171 # Nhà 2 tầng không hầm
    if ham:
        return 9504604 # Nhà hơn 2 tầng, có hầm
    return 8221171 # Nhà hơn 2 tầng, không hầm

In [52]:
df['construction_cost'] = df.apply(extract_construction_cost, axis = 1)
df['construction_cost'].value_counts()

construction_cost
8221171     15569
6275876      1442
4000000       826
9504604       337
10510920      336
12848184       26
Name: count, dtype: int64

In [53]:
have_biet_thu = df[df['description'].str.lower().str.contains('biệt thự') | df['description'].str.lower().str.contains('villa') |df['title'].str.lower().str.contains('biệt thự') |df['title'].str.lower().str.contains('villa')]
have_biet_thu.shape[0]

791

In [55]:
have_biet_thu['construction_cost'].value_counts()

construction_cost
10510920    336
8221171     290
6275876      65
4000000      52
12848184     26
9504604      22
Name: count, dtype: int64